<a href="https://colab.research.google.com/github/idan424/Electrophysiology_of_memory/blob/master/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# {M} Initialization


In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=False)

Mounted at /content/drive


# {Il} Ilya's Path

In [ ]:
PATH = '/content/drive/My Drive/Electrophysiology of the memory/Datasets/Uncompressed/'

# {Id}Idan's Path

In [2]:
PATH = '/content/drive/My Drive/Colab Notebooks/Electrophisyology of Memory/Datasets/Uncompressed/'


# {M} Imports

In [3]:
import sys
sys.path.append(PATH)
from CMLLoad import CMLLoad
load = CMLLoad(PATH)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

!pip install mne --upgrade
import mne
from mne.filter import notch_filter as notch
from mne.time_frequency import tfr_morlet as morl
from mne.time_frequency import tfr_array_morlet as amorl
mne.set_log_level(False)

from sklearn import linear_model

import warnings
warnings.filterwarnings("ignore") # needed for matplotlib warnings supression @2.3

%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['text.color'] = 'white'
plt.rcParams['xtick.color'] = 'white'
plt.rcParams['ytick.color'] = 'white'
plt.rcParams['axes.labelcolor'] = 'white'

     |████████████████████████████████| 6.8MB 2.8MB/s 


# {O} Load Patients

In [ ]:
index_df = load.Index()
subjects = index_df.subject.unique()
exp='FR1'
# print(subjects)

In [ ]:
# this part gets all the areas that have data for every subject
area_by_subj = {}
for sub in subjects:
    df_select = index_df[(index_df['subject']==sub) & (index_df['experiment']==exp)]
    if not df_select.empty:
        channels_df = load.Load(df_select.iloc[0], 'channels')
        area_by_subj[sub] = channels_df['ind.region'].dropna().unique().tolist()

# this part gets all the subjects that have data from specific area
subj_by_area = {}
for key, value in area_by_subj.items():
    for v in value:
        if v not in subj_by_area.keys(): subj_by_area[v] = [key]
        else: subj_by_area[v].append(key)


In [ ]:
# this cell shows all of the subjects that have electrodes in a specific area
for k,v in subj_by_area.items(): print(f'{k} : {subj_by_area[k]}') 

insula : ['R1380D', 'R1377M', 'R1065J', 'R1385E', 'R1189M', 'R1390M', 'R1391T', 'R1401J', 'R1361C', 'R1060M', 'R1350D', 'R1378T', 'R1375C', 'R1383J', 'R1354E', 'R1292E']
superiortemporal : ['R1380D', 'R1111M', 'R1332M', 'R1065J', 'R1385E', 'R1390M', 'R1391T', 'R1401J', 'R1361C', 'R1060M', 'R1350D', 'R1378T', 'R1375C', 'R1383J', 'R1354E', 'R1292E']
middletemporal : ['R1380D', 'R1111M', 'R1332M', 'R1377M', 'R1065J', 'R1385E', 'R1390M', 'R1391T', 'R1401J', 'R1361C', 'R1060M', 'R1350D', 'R1378T', 'R1375C', 'R1383J', 'R1354E', 'R1292E']
fusiform : ['R1380D', 'R1111M', 'R1377M', 'R1065J', 'R1385E', 'R1390M', 'R1391T', 'R1361C', 'R1350D', 'R1378T', 'R1375C', 'R1383J', 'R1354E', 'R1292E']
medialorbitofrontal : ['R1380D', 'R1065J', 'R1391T', 'R1378T', 'R1354E']
rostralmiddlefrontal : ['R1380D', 'R1377M', 'R1065J', 'R1385E', 'R1189M', 'R1391T', 'R1401J', 'R1060M', 'R1350D', 'R1378T', 'R1375C', 'R1383J', 'R1354E']
superiorfrontal : ['R1380D', 'R1377M', 'R1385E', 'R1189M', 'R1401J', 'R1361C', 'R10

# {M} Processing functions


> {M} EEG loader

In [4]:
# This function gets the subject and experiment names, and returns a generator
# The generator will yield an mne.Epochs instance for every call 
default_buf = 1000
def load_eeg(index_df, sub, exp, buf=default_buf):
    df_select = index_df[(index_df['subject']==sub) & 
                         (index_df['experiment']==exp)]
    num_elements = len(df_select)

    if df_select.empty: 
        print(f'No matching {exp} experiments for subject {sub} (empty DataFrame)\n')
        return None, None
    print(f'found {num_elements} matching {exp} experiments for subject {sub}')

    def eeg_generator(n=num_elements, buf=buf):
        for i in range(n):
            yield load.LoadMNE(df_select.iloc[i], ev_start=-buf, ev_len=buf+1000+buf), load.Load(df_select.iloc[i], 'events') 

  
    return eeg_generator(), load.Load(df_select.iloc[0], 'channels')

> {M} Notch Filter

In [5]:
# filter the signal
def apply_notch_filters(data, freqs, sr):
    for freq in freqs:
        print('[Start filtering - {}Hz]'.format(freq))
        eeg_filtered = notch(data, sr, freq, notch_widths=2, method='iir', 
                             verbose=False)
        print('[Done filtering - {}Hz]'.format(freq))

    # this has to return sampling rate since we return the np.array and not the mne.Epochs 
    return eeg_filtered

> {M} Wavelet transform



In [6]:
# this function divides returns the wanted electrode indices
def get_channel_indices(all_channels, names=None):
    if names: return [i for i,ch in enumerate(all_channels) if ch in names]
    return range(channels_df['ind.region'].shape[0])

num_elements = 30
default_freqs = np.geomspace(3, 150, num=num_elements) 
default_n_cycles = np.geomspace(1.5, 24, num=num_elements)

def get_log_morlet_generator(eeg_data, sr, electrode_chunks, freqs=default_freqs, 
                             n_cycles=default_n_cycles, buf=default_buf):
    def generator():
        for chunk in electrode_chunks:
            print(f'[Start WT, chunk - {chunk}]')
            power_data = amorl(eeg_data[:,chunk], sr, freqs=freqs, 
                            n_cycles=n_cycles, output='power')
            # print('[WT finished]')

            mean_power_data = power_data.mean(axis=0)[:,:,int(buf/1000.*sr):-1*int(buf/1000.*sr)]
            del power_data

            # yield log transform of the mean power
            yield np.log10(mean_power_data)

    return generator()



> {M} Regression 

In [7]:
# Fit a regression of log(power) ∼ log(freq) for each electrode 
# and obtain an estimate for alpha, where alpha 
# is the exponent in power ∼ 1 / freq^alpha.

def get_alphas(power_data, freqs=default_freqs):

    log_freqs = np.log10(freqs)
    n_elec = power_data.shape[0]
    alpha_est = np.zeros((n_elec,1))
    # b_est = np.zeros((n_elec,1))

    for i in range(n_elec):
        log_power = power_data[i].mean(axis=1)
        reg = linear_model.LinearRegression()
        data = pd.DataFrame({'log_pow':log_power, 'log_frequency': log_freqs})
        X = data['log_frequency'].values.reshape((len(data['log_pow']),1))
        y = data['log_pow'].values
        model = reg.fit(X, y)
        alpha_est[i] = model.coef_[0]
        # b_est[i] = model.intercept_
    
    return alpha_est

> {M} Process 1 session of a subject

In [8]:
def session_to_alpha(epochs, events):
    sr = epochs.info['sfreq']

    word_events = events[events['type']=='WORD']
    word_idx = word_events.index
    recalled_idx = word_events[word_events['recalled']==1].index
    not_recalled_idx = word_events[word_events['recalled']==0].index

    recalled_data = np.nan_to_num(epochs.get_data())[recalled_idx]
    not_recalled_data = np.nan_to_num(epochs.get_data())[not_recalled_idx]
    # print(recalled_data.shape)
    # print(not_recalled_data.shape)

    ################################################################################

    # We'd probably want to notch [60,120] at least
    filtering_freqs = [60]

    rec_filtered_eeg = apply_notch_filters(recalled_data, filtering_freqs, sr)
    n_rec_filtered_eeg = apply_notch_filters(not_recalled_data, filtering_freqs, sr)
    # print(rec_filtered_eeg.shape)
    # print(n_rec_filtered_eeg.shape)

    ################################################################################

    # electrodes should be a specific list per patient (i suggest dict((sub, region_names)))
    # names = elec_name_per_subj_dict[sub] # a list of area names
    electrodes = get_channel_indices(channels_df['ind.region'])

    chunk_size = 10
    # this condition prevents an addition of a null range
    if ~len(electrodes) % chunk_size:
        chunks = [electrodes[chunk_size*i:chunk_size*(i+1)] for i in range(int(len(electrodes)/chunk_size))]
    else:
        chunks = [electrodes[chunk_size*i:chunk_size*(i+1)] for i in range(int(len(electrodes)/chunk_size)+1)]
    # chunks.append(electrodes[chunk_size*i:-1])

    rec_log_morlet_eeg = get_log_morlet_generator(rec_filtered_eeg, sr, chunks)
    n_rec_log_morlet_eeg = get_log_morlet_generator(n_rec_filtered_eeg, sr, chunks)

    # iterating over data to get recalled and not recalled alpha values
    rec_alphas_dict = {}
    for elecs, rec_lmeeg in zip(chunks, rec_log_morlet_eeg):
        rec_alphas = [i[0] for i in get_alphas(rec_lmeeg).tolist()]
        # print(*zip(channels_df['ind.region'].iloc[elecs], rec_alphas))
        for ch_name, alpha in zip(channels_df['ind.region'].iloc[elecs], rec_alphas):
            if not ch_name in rec_alphas_dict.keys(): rec_alphas_dict[ch_name] = [alpha]
            else: rec_alphas_dict[ch_name].append(alpha)

    for k,v in rec_alphas_dict.items(): rec_alphas_dict[k] = np.array(v)
    # print(rec_alphas_dict)

    n_rec_alphas_dict = {}
    for elecs, n_rec_lmeeg in zip(chunks, n_rec_log_morlet_eeg):
        n_rec_alphas =  [i[0] for i in get_alphas(n_rec_lmeeg).tolist()]
        # print(*zip(channels_df['ind.region'].iloc[elecs], n_rec_alphas))
        for ch_name, alpha in zip(channels_df['ind.region'].iloc[elecs], n_rec_alphas):
            if not ch_name in n_rec_alphas_dict.keys(): n_rec_alphas_dict[ch_name] = [alpha]
            else: n_rec_alphas_dict[ch_name].append(alpha)

    for k,v in n_rec_alphas_dict.items(): n_rec_alphas_dict[k] = np.array(v)        
    # print(n_rec_alphas_dict)

    ################################################################################

    rec_alphas_df = pd.DataFrame()
    for k,v in rec_alphas_dict.items():
        rec_alphas_df = rec_alphas_df.append({'area':k, 'count':len(v), 
                                            'mean':v.mean(), 'std':v.std()}, 
                                            ignore_index=True)

    rec_alphas_df = rec_alphas_df.set_index('area')
    # rec_alphas_df

    n_rec_alphas_df = pd.DataFrame()
    for k,v in n_rec_alphas_dict.items():
        n_rec_alphas_df = n_rec_alphas_df.append({'area':k, 'count':len(v), 
                                                'mean':v.mean(), 'std':v.std()}, 
                                                ignore_index=True)

    n_rec_alphas_df = n_rec_alphas_df.set_index('area')
    # n_rec_alphas_df

    ################################################################################

    # combine results for recalled and not recalled 
    alpha_df = rec_alphas_df.join(n_rec_alphas_df, lsuffix='_recalled', rsuffix='_not_recalled')
    alpha_df = alpha_df.rename(columns={'count_recalled':'count'}, inplace=False)
    alpha_df = alpha_df.drop('count_not_recalled', axis=1)
    ks = alpha_df.keys()
    alpha_df = alpha_df[[ks[0], ks[1], ks[3], ks[2], ks[4]]]
    return alpha_df

# {M} Process all session for 1 subject

In [9]:
index_df = load.Index()
subjects = index_df.subject.unique()
exp = 'FR1'
subject_number = 13

# will eventually be - for sub in subjects:
sub = subjects[subject_number]
df_select = index_df[(index_df['subject']==sub) & (index_df['experiment']==exp)]
eeg_gen, channels_df = load_eeg(index_df, sub, exp)

################################################################################
try:
    session_alphas = [session_to_alpha(*next(eeg_gen))]
    # print(session_alphas[0])
    for ep, ev in eeg_gen:
        session_alphas = [*session_alphas, session_to_alpha(ep, ev)]
        
    all_alphas = {sub:session_alphas}
except Exception as e:
    pass


print(f'number of sessions - {len(all_alphas[sub])}')
all_alphas[sub][0]

found 4 matching FR1 experiments for subject R1060M
[Start filtering - 60Hz]
[Done filtering - 60Hz]
[Start filtering - 60Hz]
[Done filtering - 60Hz]
[Start WT, chunk - range(0, 10)]
[Start WT, chunk - range(10, 20)]
[Start WT, chunk - range(20, 30)]
[Start WT, chunk - range(30, 40)]
[Start WT, chunk - range(40, 50)]
[Start WT, chunk - range(50, 60)]
[Start WT, chunk - range(60, 70)]
[Start WT, chunk - range(70, 80)]
[Start WT, chunk - range(0, 10)]
[Start WT, chunk - range(10, 20)]
[Start WT, chunk - range(20, 30)]
[Start WT, chunk - range(30, 40)]
[Start WT, chunk - range(40, 50)]
[Start WT, chunk - range(50, 60)]
[Start WT, chunk - range(60, 70)]
[Start WT, chunk - range(70, 80)]
[Start filtering - 60Hz]
[Done filtering - 60Hz]
[Start filtering - 60Hz]
[Done filtering - 60Hz]
[Start WT, chunk - range(0, 10)]
[Start WT, chunk - range(10, 20)]
[Start WT, chunk - range(20, 30)]
[Start WT, chunk - range(30, 40)]
[Start WT, chunk - range(40, 50)]
[Start WT, chunk - range(50, 60)]
[Start 

,count,mean_recalled,mean_not_recalled,std_recalled,std_not_recalled
area,,,,,
middletemporal,19.0,-4.203527,-4.207331,0.555653,0.562014
inferiortemporal,3.0,-5.378416,-5.375158,0.388078,0.399129
lateralorbitofrontal,1.0,-4.733167,-4.753555,0.000000,0.000000
insula,3.0,-4.492077,-4.510656,0.802036,0.777087
superiorfrontal,3.0,-4.010957,-4.024810,0.406829,0.403522
frontalpole,1.0,-4.877803,-4.889129,0.000000,0.000000
rostralmiddlefrontal,13.0,-4.226109,-4.217245,0.368900,0.368186
parsorbitalis,2.0,-3.941998,-3.945931,0.090271,0.088416
inferiorparietal,2.0,-3.195210,-3.177907,0.021616,0.007402


# {P} Pickling stage - alpha for patient(all sessions) dict's

all_sessions is dict(key=subject_name, value=list(pd.DF))

In [10]:
import pickle

to pickle, get byte code, and replicate the code in the other cells at the bottom of the NB

In [11]:
sub

R1060M
b'\x80\x03}q\x00X\x06\x00\x00\x00R1060Mq\x01]q\x02(cpandas.core.frame\nDataFrame\nq\x03)\x81q\x04}q\x05(X\x04\x00\x00\x00_mgrq\x06cpandas.core.internals.managers\nBlockManager\nq\x07)\x81q\x08(]q\t(cpandas.core.indexes.base\n_new_Index\nq\ncpandas.core.indexes.base\nIndex\nq\x0b}q\x0c(X\x04\x00\x00\x00dataq\rcnumpy.core.multiarray\n_reconstruct\nq\x0ecnumpy\nndarray\nq\x0fK\x00\x85q\x10C\x01bq\x11\x87q\x12Rq\x13(K\x01K\x05\x85q\x14cnumpy\ndtype\nq\x15X\x02\x00\x00\x00O8q\x16K\x00K\x01\x87q\x17Rq\x18(K\x03X\x01\x00\x00\x00|q\x19NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?tq\x1ab\x89]q\x1b(X\x05\x00\x00\x00countq\x1cX\r\x00\x00\x00mean_recalledq\x1dX\x11\x00\x00\x00mean_not_recalledq\x1eX\x0c\x00\x00\x00std_recalledq\x1fX\x10\x00\x00\x00std_not_recalledq etq!bX\x04\x00\x00\x00nameq"Nu\x86q#Rq$h\nh\x0b}q%(h\rh\x0eh\x0fK\x00\x85q&h\x11\x87q\'Rq((K\x01K\x0f\x85q)h\x18\x89]q*(X\x0e\x00\x00\x00middletemporalq+X\x10\x00\x00\x00inferiortemporalq,X\x14\x00\x00\x00lateralorbitofrontalq-X\x06\x0

In [ ]:
pickle.dumps(all_alphas)

# {U} Unpickling - choose byte_alpha first

Unpickling will be stupid for now, but will improve later on

In [ ]:
all_alphas = pickle.loads(byte_alpha)

This will show you the firt session alphas for the loaded patient

In [ ]:
sub = [*all_alphas.keys()][0]
all_alphas[sub][0]

# {Arc} Pickled alpha for patient (all sessions) dict's

all_sessions is a dict(key=subject_name,value=list(pd.DF)) for each subject:

first cell is a template

---

**R_____** - 


In [ ]:
byte_alpha = b''

---

**R1380D** - 0

In [ ]:
byte_alpha = b'\x80\x03}q\x00X\x06\x00\x00\x00R1380Dq\x01]q\x02cpandas.core.frame\nDataFrame\nq\x03)\x81q\x04}q\x05(X\x05\x00\x00\x00_dataq\x06cpandas.core.internals.managers\nBlockManager\nq\x07)\x81q\x08(]q\t(cpandas.core.indexes.base\n_new_Index\nq\ncpandas.core.indexes.base\nIndex\nq\x0b}q\x0c(X\x04\x00\x00\x00dataq\rcnumpy.core.multiarray\n_reconstruct\nq\x0ecnumpy\nndarray\nq\x0fK\x00\x85q\x10C\x01bq\x11\x87q\x12Rq\x13(K\x01K\x05\x85q\x14cnumpy\ndtype\nq\x15X\x02\x00\x00\x00O8q\x16K\x00K\x01\x87q\x17Rq\x18(K\x03X\x01\x00\x00\x00|q\x19NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?tq\x1ab\x89]q\x1b(X\x05\x00\x00\x00countq\x1cX\r\x00\x00\x00mean_recalledq\x1dX\x11\x00\x00\x00mean_not_recalledq\x1eX\x0c\x00\x00\x00std_recalledq\x1fX\x10\x00\x00\x00std_not_recalledq etq!bX\x04\x00\x00\x00nameq"Nu\x86q#Rq$h\nh\x0b}q%(h\rh\x0eh\x0fK\x00\x85q&h\x11\x87q\'Rq((K\x01K\x13\x85q)h\x18\x89]q*(X\x06\x00\x00\x00insulaq+X\x10\x00\x00\x00superiortemporalq,X\x0e\x00\x00\x00middletemporalq-G\x7f\xf8\x00\x00\x00\x00\x00\x00X\x08\x00\x00\x00fusiformq.X\x13\x00\x00\x00medialorbitofrontalq/X\x14\x00\x00\x00rostralmiddlefrontalq0X\x0f\x00\x00\x00superiorfrontalq1X\n\x00\x00\x00entorhinalq2X\x10\x00\x00\x00inferiortemporalq3X\x08\x00\x00\x00banksstsq4X\x0b\x00\x00\x00postcentralq5X\n\x00\x00\x00precentralq6X\x13\x00\x00\x00caudalmiddlefrontalq7X\x10\x00\x00\x00parstriangularisq8X\x0b\x00\x00\x00paracentralq9X\x12\x00\x00\x00posteriorcingulateq:X\x18\x00\x00\x00rostralanteriorcingulateq;X\x14\x00\x00\x00lateralorbitofrontalq<etq=bh"X\x04\x00\x00\x00areaq>u\x86q?Rq@e]qAh\x0eh\x0fK\x00\x85qBh\x11\x87qCRqD(K\x01K\x05K\x13\x86qEh\x15X\x02\x00\x00\x00f8qFK\x00K\x01\x87qGRqH(K\x03X\x01\x00\x00\x00<qINNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tqJb\x89B\xf8\x02\x00\x00\x00\x00\x00\x00\x00\x00,@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x005@\x00\x00\x00\x00\x00\x003@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x002@\x00\x00\x00\x00\x00\x00A@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00.@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x10@\x1bhyo\xeaq\x06\xc0\xcf\xc0\t\xa9p\x0e\t\xc0\xe7\xcc-\xae\xc9\xe6\x07\xc0\xe3\\u\x00\xec\xa1\x06\xc0k\x9aS\x05\xed(\x05\xc0>\xaf\xecV\xa1\xdf\x07\xc0?\xe6\xcf\xe4c\xde\x06\xc0\xb8\xeb\xe2\x10C)\x07\xc0\x10\xf7\xdf\x10\xe0\xdf\x04\xc0h1\x0b\xcb\x13\t\t\xc0\xc8h\x1a\xdc\x94\x8a\x04\xc0&!)\x82P\x1c\x07\xc0\xf9\\]L\x06\xa1\x06\xc0?\xc9\x1c\xafF\t\x04\xc0W\x9b\x13$\x14\x0f\t\xc0\x1aC\xa9\x8d\xfd%\x06\xc0\xba\xf9\x9d\xee\xe7\x9b\x05\xc0\x13\xcbe\xf9#m\x08\xc0\x95\\r4!\xe9\t\xc0m\xae\xd5pS}\x06\xc0\xd0\x9b\xe6\x88=\x01\t\xc0r=\x9e\xaaA\x03\x08\xc0\x06\x9d\x13\x12\x8a\x95\x06\xc0}\xf0\xb7\x1c|Z\x05\xc0\x90\x023c0\xe3\x07\xc0T?(\x90\x9d\xde\x06\xc0_\xd6\xf0Tn\'\x07\xc0\xc0\xf7\x17\xcc\n\x18\x05\xc0\xa7\x9dXL90\t\xc0}\x9co\x05j\xdf\x04\xc0mg\n\xcf\xde\x07\x07\xc0m\x8c\x81\xde\xb1\x91\x06\xc0\x14?\xeei:\xf9\x03\xc0\xe8\x92\x9b\xdc9-\t\xc0\\\xe6\xcd*\xb5\x1a\x06\xc0\x13i[\x9e9\xc1\x05\xc05Q\'\xd6\xf1\x82\x08\xc0\x97\x9b\x00\xc7\xab\xf6\t\xc0@\xe5\xcd\x83\x92\xca\xcf?dp\x12\xd1jr\xc4?h\x190\xc5\xedq\xd2?\x8dNb^\x10\x10\xc8?\xfe\x1fY5\xc1\xb6\xd2?u\xae\x08\x19\xb2;\xc7?\xa3%\xae%)O\xd0?\x8a\xfe\xf0\x85\xae\xac\xcf?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc9\x00^`p\x1a\xd3?\x11\xc9\x1c\x96ms\xc6?\xd0gtI\xcd\xa0\xc7?\xa6=\xb3?\xb3\x82\xcb?[,\xbc\xda\xbc\x88\xce?\xfc\xbd\xe7?\xe2D\xd3?\x00\x00\x00\x00\x00\x00\x00\x00\xdaj\x12/B\xb4\xc2?\x04:\xe4\n\xabc\xce?\x94%\xf5\x19\x9c\xd9\xc5?\x9b\x84a\xcb\xd7n\xd2?hC\xb5\x17\xc3\xdc\xc8?\x99\x8a\xc5\xf6\x91\x02\xd2?\\\x07\xe2\xa2\x1d\xb0\xc8?L\xc2\x9f3\x85\xdf\xcb?g\xfa\xcdr\xbf6\xd0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x85\x1a\xb1\xe1\x93\xc6\xd1?\xad\xac\x87\xdd\xd2\x10\xc6?\xd0\x18~2W\xcb\xc8?\xb1\x88\xd2Z\xb8O\xc9?u\x9f\x03(\x80\x8c\xce?\xa8\xbd\x07\xe8~=\xd3?\x00\x00\x00\x00\x00\x00\x00\x00j\xea\x06FU\xbd\xc2?qKtqLba]qMh\nh\x0b}qN(h\rh\x0eh\x0fK\x00\x85qOh\x11\x87qPRqQ(K\x01K\x05\x85qRh\x18\x89]qS(h\x1ch\x1dh\x1eh\x1fh etqTbh"Nu\x86qURqVa}qWX\x06\x00\x00\x000.14.1qX}qY(X\x04\x00\x00\x00axesqZh\tX\x06\x00\x00\x00blocksq[]q\\}q](X\x06\x00\x00\x00valuesq^hDX\x08\x00\x00\x00mgr_locsq_cbuiltins\nslice\nq`K\x00K\x05K\x01\x87qaRqbuaustqcbX\x04\x00\x00\x00_typqdX\t\x00\x00\x00dataframeqeX\t\x00\x00\x00_metadataqf]qgX\x05\x00\x00\x00attrsqh}qiubas.'

---

**R1111M** - 1


In [ ]:
byte_alpha = b'\x80\x03}q\x00X\x06\x00\x00\x00R1111Mq\x01]q\x02(cpandas.core.frame\nDataFrame\nq\x03)\x81q\x04}q\x05(X\x05\x00\x00\x00_dataq\x06cpandas.core.internals.managers\nBlockManager\nq\x07)\x81q\x08(]q\t(cpandas.core.indexes.base\n_new_Index\nq\ncpandas.core.indexes.base\nIndex\nq\x0b}q\x0c(X\x04\x00\x00\x00dataq\rcnumpy.core.multiarray\n_reconstruct\nq\x0ecnumpy\nndarray\nq\x0fK\x00\x85q\x10C\x01bq\x11\x87q\x12Rq\x13(K\x01K\x05\x85q\x14cnumpy\ndtype\nq\x15X\x02\x00\x00\x00O8q\x16K\x00K\x01\x87q\x17Rq\x18(K\x03X\x01\x00\x00\x00|q\x19NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?tq\x1ab\x89]q\x1b(X\x05\x00\x00\x00countq\x1cX\r\x00\x00\x00mean_recalledq\x1dX\x11\x00\x00\x00mean_not_recalledq\x1eX\x0c\x00\x00\x00std_recalledq\x1fX\x10\x00\x00\x00std_not_recalledq etq!bX\x04\x00\x00\x00nameq"Nu\x86q#Rq$h\nh\x0b}q%(h\rh\x0eh\x0fK\x00\x85q&h\x11\x87q\'Rq((K\x01K\x0e\x85q)h\x18\x89]q*(X\x0e\x00\x00\x00middletemporalq+X\x10\x00\x00\x00superiortemporalq,X\x0b\x00\x00\x00postcentralq-X\r\x00\x00\x00supramarginalq.X\x08\x00\x00\x00banksstsq/X\x10\x00\x00\x00inferiortemporalq0X\x10\x00\x00\x00inferiorparietalq1X\x08\x00\x00\x00fusiformq2X\x10\x00\x00\x00lateraloccipitalq3X\x06\x00\x00\x00cuneusq4X\t\x00\x00\x00precuneusq5X\x10\x00\x00\x00superiorparietalq6X\x0c\x00\x00\x00temporalpoleq7X\x0f\x00\x00\x00parahippocampalq8etq9bh"X\x04\x00\x00\x00areaq:u\x86q;Rq<e]q=h\x0eh\x0fK\x00\x85q>h\x11\x87q?Rq@(K\x01K\x05K\x0e\x86qAh\x15X\x02\x00\x00\x00f8qBK\x00K\x01\x87qCRqD(K\x03X\x01\x00\x00\x00<qENNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tqFb\x89B0\x02\x00\x00\x00\x00\x00\x00\x00\x002@\x00\x00\x00\x00\x00\x000@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00>@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00,@\x00\x00\x00\x00\x00\x006@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00*@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00@.\x03\xa5)\xaa\xb3\x11\xc0K&\x13X5:\x12\xc0\xd5\xa2\\\xeb3\x7f\x13\xc0|\xd8\x167\x96|\x12\xc0If\x00\xe4a\xed\x12\xc0\xc8\xcb(\xce\xc7\x97\x11\xc0T\x92\x9e\x04B\xaf\x13\xc0]X\t8E\'\x11\xc0|\x1d\xe8\xab\xaa\xe6\x10\xc0yoG\xcd\xc5\xd5\x14\xc0-S\x0e\xa5\xc9|\x12\xc0}\xd3\xe3\xbc$\x95\x12\xc0F\x93\xf7z\xd4\x86\x10\xc0\x1a\x81\xeba\xb2\xa1\x14\xc0i\xf2\xb7\xd2\xe5\xce\x11\xc09]\xbc\x0c\x1eQ\x12\xc0\xa8\xc9\x04\xd2\xd5\x8c\x13\xc0\x15Xy\xee,\x8f\x12\xc0C:\x81\xcf>\x07\x13\xc0jS\x9c\x08\xcb\xc9\x11\xc0Jx\xb1\x11A\xbc\x13\xc0\x18@\x19\xf7\x10J\x11\xc0|\xd8\xf1\xe5\x01\x01\x11\xc0\xf7\xec0\xc9\xb0\xe4\x14\xc0\xe4\x9a\x89\x86\x12u\x12\xc0+\x16\\n\x0e\x96\x12\xc0\xff\x1f=\x9bT\x9b\x10\xc0\xcc\xf0\xef\xd2\xa3\xc0\x14\xc0\x1cm/\xd19\xa0\xe0?l\xbcK\xfe5\xeb\xdd?\xb9\xa4\xa8\x19\x82U\xd2?\xd2Z?\xf9\xef\x1c\xdb?\x14\xe7\xb6\xd6\xbc\xbb\xd4?\xea\xc7\xc2\x1a"4\xe3?\xe3G\xd8J\xd0\xf7\xe3?\xde\xfd_\xf70\xe5\xdb?)\xa30\\\xb0\x90\xda?\x00\x00\x00\x00\x00\x00\x00\x00`\xe5\x8e0I\xc2\xe1?*\xa5`\x85\xa8\xc7\xeb?\xd8\xf8\xb8\xcb`\xea\xd9? \x08\xd2\xa8\x7f\x8c\xbe?*\xd3\x16\x9d\xfbA\xe0?r\xa9\x1eG\xadu\xdd?\x88\xc1;\xe2\x88\xa6\xd1?\x1f\xbeD\x10j\x0e\xdb?7\xc5\xf2\xf4\x1a\x95\xd4?1li\x87T\xe3\xe1?9x\x96\xdeJ\xaf\xe3?7\x98:\xcd\xc8\xbc\xdb?\x1b\xba\x15f\xd6b\xdb?\x00\x00\x00\x00\x00\x00\x00\x00\xa8q\x16\xe9\xb9\xd4\xe1?\xda\xd8h\x9b\xe4\xd9\xeb?P|P:e\xcd\xd9?`\xbcl\xfe\xdb"\xbf?qGtqHba]qIh\nh\x0b}qJ(h\rh\x0eh\x0fK\x00\x85qKh\x11\x87qLRqM(K\x01K\x05\x85qNh\x18\x89]qO(h\x1ch\x1dh\x1eh\x1fh etqPbh"Nu\x86qQRqRa}qSX\x06\x00\x00\x000.14.1qT}qU(X\x04\x00\x00\x00axesqVh\tX\x06\x00\x00\x00blocksqW]qX}qY(X\x06\x00\x00\x00valuesqZh@X\x08\x00\x00\x00mgr_locsq[cbuiltins\nslice\nq\\K\x00K\x05K\x01\x87q]Rq^uaustq_bX\x04\x00\x00\x00_typq`X\t\x00\x00\x00dataframeqaX\t\x00\x00\x00_metadataqb]qcX\x05\x00\x00\x00attrsqd}qeubh\x03)\x81qf}qg(h\x06h\x07)\x81qh(]qi(h\nh\x0b}qj(h\rh\x0eh\x0fK\x00\x85qkh\x11\x87qlRqm(K\x01K\x05\x85qnh\x18\x89]qo(h\x1cX\r\x00\x00\x00mean_recalledqpX\x11\x00\x00\x00mean_not_recalledqqX\x0c\x00\x00\x00std_recalledqrX\x10\x00\x00\x00std_not_recalledqsetqtbh"Nu\x86quRqvh\nh\x0b}qw(h\rh\x0eh\x0fK\x00\x85qxh\x11\x87qyRqz(K\x01K\x0e\x85q{h\x18\x89]q|(h+h,h-h.h/h0h1h2h3h4h5h6h7h8etq}bh"h:u\x86q~Rq\x7fe]q\x80h\x0eh\x0fK\x00\x85q\x81h\x11\x87q\x82Rq\x83(K\x01K\x05K\x0e\x86q\x84hD\x89B0\x02\x00\x00\x00\x00\x00\x00\x00\x002@\x00\x00\x00\x00\x00\x000@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00>@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00,@\x00\x00\x00\x00\x00\x006@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00*@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00@0C\xf2\xb1\x8f\x80\x11\xc0\x96\xa0#\x83\x0e\xba\x11\xc0\xf0IS\xb6\xc4\x83\x13\xc0\xe5\xd8\xf5e\xc9#\x12\xc0F7\xb4L\xffo\x12\xc0F\xbf\xbdR\xcbf\x11\xc0\xcc\xc1G\xe5\xc0\xa0\x13\xc0\xc0pm\xfbq@\x12\xc0\xef\xe6s\xe5\x86M\x12\xc0[V\xd8\xe0,\xed\x14\xc0r\xe6\x8e\xc8lg\x12\xc0\xd7s\xf2\xfd\x03\x7f\x12\xc0\x94\x9e9\xbb\x11\xaf\x10\xc0)\xc3\xb5&\xd6\x8e\x14\xc06\xe5\x99J\xa8\x85\x11\xc0]D3\xab\xcb\xc4\x11\xc0\xe3\xc7\xben\xba\x96\x13\xc0\x91j\x19\xba7/\x12\xc0\xc8\xe1\x91\x9b\x84w\x12\xc0\xad+Rj\xf2\x8a\x11\xc0S\x13\xf8+>\xaa\x13\xc0\x1e\xe1\xfeiHW\x12\xc0\x9bW\xf5\x0cx_\x12\xc0\x01\xd0k\xb5\x8a\xf3\x14\xc0\xd7vF\x07\x8b^\x12\xc0\xbeA\xc4\xe8\xf2\x86\x12\xc0\x0e\xac\xab\xac$\xb5\x10\xc0\xd27Cu\x01\x9d\x14\xc0\xf9 \xc9\x14\x1b\xb8\xe0?\x94\x8a\xbe\x01r\x1a\xdf?\x99\x92\xd4\xd4I\xea\xd2?\xdc\xe9\xfa\rR[\xdb?O\xb9\xf4\x11\xc9\xcc\xd6?a\xd7\xfd\xda\x85\x05\xe4?wfw\xd8"R\xe5?\xbb\xb12I\x9e;\xe2?\x08\x90\xbf\x83A\x03\xe5?\x00\x00\x00\x00\x00\x00\x00\x00\x843\xc5+\x17\xb8\xe1?\xfe\xe9\r\x99\xee\x8c\xec?\xd4\x18\x1b#\xc3\x96\xd9?\xc0\xb3P\xc6\x1b\x8e\xbf?~\xa7\xb2j[\xd5\xe0?kF\x8cj[\xf3\xde?"\x16n!\xfd\x95\xd2?\xfb\xee\x0c\xee\x9c\xe3\xda?\x1fcYx\x88\xce\xd6?{~c*\x8f\r\xe3?T\xdd\xb9F\xbdT\xe5?n\x80[\x9dp\x80\xe2?\x8c5\t >\xcd\xe4?\x00\x00\x00\x00\x00\x00\x00\x00\xf8%\xab\x17\x96\xab\xe1?\xa7\xe7\x18\x9c`i\xec?\xf8!\xf3\xa1\\\xe1\xd9?@\x17\xb0\x94\x01\x06\xbf?q\x85tq\x86ba]q\x87h\nh\x0b}q\x88(h\rh\x0eh\x0fK\x00\x85q\x89h\x11\x87q\x8aRq\x8b(K\x01K\x05\x85q\x8ch\x18\x89]q\x8d(h\x1chphqhrhsetq\x8ebh"Nu\x86q\x8fRq\x90a}q\x91hT}q\x92(hVhihW]q\x93}q\x94(hZh\x83h[h\\K\x00K\x05K\x01\x87q\x95Rq\x96uaustq\x97bh`hahbhchd}q\x98ubh\x03)\x81q\x99}q\x9a(h\x06h\x07)\x81q\x9b(]q\x9c(h\nh\x0b}q\x9d(h\rh\x0eh\x0fK\x00\x85q\x9eh\x11\x87q\x9fRq\xa0(K\x01K\x05\x85q\xa1h\x18\x89]q\xa2(h\x1cX\r\x00\x00\x00mean_recalledq\xa3X\x11\x00\x00\x00mean_not_recalledq\xa4X\x0c\x00\x00\x00std_recalledq\xa5X\x10\x00\x00\x00std_not_recalledq\xa6etq\xa7bh"Nu\x86q\xa8Rq\xa9h\nh\x0b}q\xaa(h\rh\x0eh\x0fK\x00\x85q\xabh\x11\x87q\xacRq\xad(K\x01K\x0e\x85q\xaeh\x18\x89]q\xaf(h+h,h-h.h/h0h1h2h3h4h5h6h7h8etq\xb0bh"h:u\x86q\xb1Rq\xb2e]q\xb3h\x0eh\x0fK\x00\x85q\xb4h\x11\x87q\xb5Rq\xb6(K\x01K\x05K\x0e\x86q\xb7hD\x89B0\x02\x00\x00\x00\x00\x00\x00\x00\x002@\x00\x00\x00\x00\x00\x000@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00>@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00,@\x00\x00\x00\x00\x00\x006@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00*@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00@\x1f\x9cd\x02\x83x\x11\xc0\xb6\xc8\xf76\x06D\x12\xc0\x00\'\xed\x8b\xeb\xb7\x13\xc0I\x95\x8b\x9fE(\x12\xc0\xebZ\x93\xea\xdc\x1c\x13\xc0\rH\xc2h\xbd\x88\x11\xc0\xe5?\xa7;\x19;\x13\xc0\xca\xde\xb1\xb8\xe0.\x12\xc0Y\xef\xa8I\x02\x0f\x12\xc0\xd2\xa0\xa5\xd8\xd2c\x14\xc0\xe4\x01\x95\x1e\xdd\xd6\x11\xc0b\xb9\x00\xa0\x88%\x12\xc0\xd7h\x94{I\xd7\x0f\xc0\xf5\x941\x00\x93T\x14\xc0\xcf]1A\xfa\x8f\x11\xc0\xf9ow\xe6We\x12\xc0\\\xaf\xd71\xdb\xe3\x13\xc0\x11\xfbU\xe4 D\x12\xc0\x17\x99d\xe1]K\x13\xc0\x8a\xd8\xce\x90\x9d\xb5\x11\xc0\xe13XSPC\x13\xc0m\xa2*\xffiN\x12\xc0\x84\x04\x9c\x10V)\x12\xc0L\xe1\x1e\xba,\x80\x14\xc0F#\xce"^\xf5\x11\xc0\xe9\'\xa3I\x1e5\x12\xc0Po}R\x03\xd1\x0f\xc0\x1eU0h\xf3W\x14\xc0\xcb]\x13\xe1\x89\x9f\xe3?\xb9\x8fL\xb1\x0f\xbd\xdd?B\x8fWU\xc0\xbe\xd1?\r\x9dR.\x98\x11\xdd?\xb8\\=\x01\x88\xc3\xd0?\xfa\xc1\xc9\xb9y\x14\xe5?e\x17\x8b\xa5\x8d\xcc\xe7?\x00\x18\xf6\x08\x0e|\xe4?\x96f\xa8\x88\xe9m\xeb?\x00\x00\x00\x00\x00\x00\x00\x00>\xaa+!\xd8\x8a\xe1?\x9b\x82$U\xd0v\xee?8\xc9X\x1c\xee\x96\xdb?\xa0b`\xae\xa6\xb3\xc0?\x7f\xcd\xc9\x81\xd9^\xe3?A\xc7\x1f-\x1a\x90\xdd?\x95\x88B\x8c\xfe\r\xd2?~\x1c\xa6\x83\xf5\xa9\xdc?\xa2=\xba\xcf8\xf2\xd0?(r\xce\xf4\x9f@\xe4?\x92>\x8aV\xe2}\xe7?\xccU\x0c*\xe6\xc0\xe4?T\xc8\tK\xc1A\xeb?\x00\x00\x00\x00\x00\x00\x00\x00\x88\x1d[d]\x8f\xe1?8\xd1N\xb4\xe1\x19\xee?\x04JW\xde\x05\xa5\xdb?`U\x9f\x15\x91\xe3\xc0?q\xb8tq\xb9ba]q\xbah\nh\x0b}q\xbb(h\rh\x0eh\x0fK\x00\x85q\xbch\x11\x87q\xbdRq\xbe(K\x01K\x05\x85q\xbfh\x18\x89]q\xc0(h\x1ch\xa3h\xa4h\xa5h\xa6etq\xc1bh"Nu\x86q\xc2Rq\xc3a}q\xc4hT}q\xc5(hVh\x9chW]q\xc6}q\xc7(hZh\xb6h[h\\K\x00K\x05K\x01\x87q\xc8Rq\xc9uaustq\xcabh`hahbhchd}q\xcbubh\x03)\x81q\xcc}q\xcd(h\x06h\x07)\x81q\xce(]q\xcf(h\nh\x0b}q\xd0(h\rh\x0eh\x0fK\x00\x85q\xd1h\x11\x87q\xd2Rq\xd3(K\x01K\x05\x85q\xd4h\x18\x89]q\xd5(h\x1cX\r\x00\x00\x00mean_recalledq\xd6X\x11\x00\x00\x00mean_not_recalledq\xd7X\x0c\x00\x00\x00std_recalledq\xd8X\x10\x00\x00\x00std_not_recalledq\xd9etq\xdabh"Nu\x86q\xdbRq\xdch\nh\x0b}q\xdd(h\rh\x0eh\x0fK\x00\x85q\xdeh\x11\x87q\xdfRq\xe0(K\x01K\x0e\x85q\xe1h\x18\x89]q\xe2(h+h,h-h.h/h0h1h2h3h4h5h6h7h8etq\xe3bh"h:u\x86q\xe4Rq\xe5e]q\xe6h\x0eh\x0fK\x00\x85q\xe7h\x11\x87q\xe8Rq\xe9(K\x01K\x05K\x0e\x86q\xeahD\x89B0\x02\x00\x00\x00\x00\x00\x00\x00\x002@\x00\x00\x00\x00\x00\x000@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00>@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00,@\x00\x00\x00\x00\x00\x006@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00*@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00@\xcd\x85\x9fZ\x11/\x11\xc0\xd2\xd7\xda\xa0\xe7\x99\x11\xc0g\xab\xc5J\xb0c\x13\xc0\xa5F\xd0khI\x11\xc0\\\x97\xa43\x05\xda\x11\xc0\xc6\x9d>\x16\x91Q\x11\xc0\xc7B\xef\xa7\x7f!\x12\xc0\xf7\x8fw\x99\xf3\xcf\x12\xc0\xee@\xec\xd8\xd8\xb4\x12\xc0\xc2\x00I8,\xfe\x13\xc0\xb4\xb0\xdb\xa4V\x08\x11\xc0JK)f\x84\x0e\x12\xc0\xe0\x0b\xe1\x11M\xed\x0f\xc0m\x93\xd8DfU\x14\xc0\xe7yJ\xf3[F\x11\xc0\x14{\x0cS\x92\xa2\x11\xc0\x83e\xe3\x14;r\x13\xc0\xa9\x19\xdf\rb\\\x11\xc0\xb8\x8c[\xafh\xed\x11\xc0r\xd1\x15\xe0Pn\x11\xc0\xc4\xdf2vK%\x12\xc0\xb6\x17\xa7ZI\xea\x12\xc0\x9a&v\x1b\xc4\xca\x12\xc0\xeab7\xe66\xfc\x13\xc0R\xf6\x92L\xc2\xf8\x10\xc0\x92\x97\xfb\xe3t\x14\x12\xc0\xf9X%\xf47\xfb\x0f\xc0V7ue\xb0f\x14\xc0\xe8\xaa\xbb\xbc\xf7\x8d\xe4?\x03N^\xdb\r\x12\xe2?(\xda\xe4i|\xbd\xd2?\xc3%\x9c\xfe\xf3\xa5\xdf?O\xd8j?b\x12\xce?:\xfb\xc28$)\xe6?E\x14;\r\xc3\xa0\xe8?\x84\xb4\xf9\xec\xf55\xe6?\xc7\xd5\x8fE\xa7\xc6\xeb?\x00\x00\x00\x00\x00\x00\x00\x00B8\xcaN\x94T\xe1?:-\x05\xe8\xef\x19\xef?,0[\xc5\xa0\xdd\xdd?\x80\xda\xaa*qw\xbc?Sm\x0eH\xe0\xa7\xe4?\x00\x16S=qD\xe2?\x0b\xb0\x88\xa4(\xed\xd2?4\xfc\x8c\x80M\xc7\xdf?\x15\x8c\x160\xa0\x11\xcd?\xd5h\xc5a`\xd9\xe5?\x91p]\x02\xb1\x8f\xe8?\xc0\xf4m]\x83v\xe6?N\xa6g\x0c\x07\xe2\xeb?\x00\x00\x00\x00\x00\x00\x00\x00\xba\x8f\xe6\x92\xacV\xe1?g\x13\x0c\xccA\xfb\xee?\x08n\xf4\x12\xac\xb5\xdd?\xe0k\x8cQ"T\xbc?q\xebtq\xecba]q\xedh\nh\x0b}q\xee(h\rh\x0eh\x0fK\x00\x85q\xefh\x11\x87q\xf0Rq\xf1(K\x01K\x05\x85q\xf2h\x18\x89]q\xf3(h\x1ch\xd6h\xd7h\xd8h\xd9etq\xf4bh"Nu\x86q\xf5Rq\xf6a}q\xf7hT}q\xf8(hVh\xcfhW]q\xf9}q\xfa(hZh\xe9h[h\\K\x00K\x05K\x01\x87q\xfbRq\xfcuaustq\xfdbh`hahbhchd}q\xfeubes.'

---

**R1332M** - 2

In [ ]:
byte_alpha = b'\x80\x03}q\x00X\x06\x00\x00\x00R1332Mq\x01]q\x02(cpandas.core.frame\nDataFrame\nq\x03)\x81q\x04}q\x05(X\x05\x00\x00\x00_dataq\x06cpandas.core.internals.managers\nBlockManager\nq\x07)\x81q\x08(]q\t(cpandas.core.indexes.base\n_new_Index\nq\ncpandas.core.indexes.base\nIndex\nq\x0b}q\x0c(X\x04\x00\x00\x00dataq\rcnumpy.core.multiarray\n_reconstruct\nq\x0ecnumpy\nndarray\nq\x0fK\x00\x85q\x10C\x01bq\x11\x87q\x12Rq\x13(K\x01K\x05\x85q\x14cnumpy\ndtype\nq\x15X\x02\x00\x00\x00O8q\x16K\x00K\x01\x87q\x17Rq\x18(K\x03X\x01\x00\x00\x00|q\x19NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?tq\x1ab\x89]q\x1b(X\x05\x00\x00\x00countq\x1cX\r\x00\x00\x00mean_recalledq\x1dX\x11\x00\x00\x00mean_not_recalledq\x1eX\x0c\x00\x00\x00std_recalledq\x1fX\x10\x00\x00\x00std_not_recalledq etq!bX\x04\x00\x00\x00nameq"Nu\x86q#Rq$h\nh\x0b}q%(h\rh\x0eh\x0fK\x00\x85q&h\x11\x87q\'Rq((K\x01K\x06\x85q)h\x18\x89]q*(G\x7f\xf8\x00\x00\x00\x00\x00\x00X\x10\x00\x00\x00superiortemporalq+X\x0e\x00\x00\x00middletemporalq,X\x18\x00\x00\x00rostralanteriorcingulateq-X\x14\x00\x00\x00lateralorbitofrontalq.X\x10\x00\x00\x00parstriangularisq/etq0bh"X\x04\x00\x00\x00areaq1u\x86q2Rq3e]q4h\x0eh\x0fK\x00\x85q5h\x11\x87q6Rq7(K\x01K\x05K\x06\x86q8h\x15X\x02\x00\x00\x00f8q9K\x00K\x01\x87q:Rq;(K\x03X\x01\x00\x00\x00<q<NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq=b\x89C\xf0\x00\x00\x00\x00\x00\x009@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00<@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00 @\xe8\x93\xf9iM}\x03\xc0A\x0b\xcf\xdc\xc2C\x05\xc0\xd3\xdfZ\xfc@\xc5\x04\xc0\x8f\xe5\x89W\x95\x00\x03\xc0A\xd6mZ\x8b\x82\x00\xc0<sQXH\xfd\x00\xc0\xb9\xd9^?\xbc}\x03\xc0\x85eR|\xe8:\x05\xc0uQ\x9cP\xa4\xee\x04\xc0\x1c?\x96H\xce\xe3\x02\xc0}\xdf\x0e\xfe>\x83\x00\xc0\xb8\xbd\xd9>%\x06\x01\xc0\x10\x8d\xcdZ{\xa5\xd1?\x04\x1c\xb9\xde\x12 \xc9?\t\xed\x02\x95\x1b*\xd6?tv<s\xe3\xde\xd4?\xf9\xe9e-K\x12\xdc?\xdc\x1f\x80\x08\x9f\t\xd5?\xa6\xd0^\xa2\x0f\n\xd2?\xa6\xae\xb1H"\x1c\xc9?\xee\x9b\x12\xdd\x90\x03\xd5?(\xd5\xc1\x82ai\xd5?\xdcO\x997\x15\'\xdc?\xf8\xa7\xfd\xbf\xcbG\xd5?q>tq?ba]q@h\nh\x0b}qA(h\rh\x0eh\x0fK\x00\x85qBh\x11\x87qCRqD(K\x01K\x05\x85qEh\x18\x89]qF(h\x1ch\x1dh\x1eh\x1fh etqGbh"Nu\x86qHRqIa}qJX\x06\x00\x00\x000.14.1qK}qL(X\x04\x00\x00\x00axesqMh\tX\x06\x00\x00\x00blocksqN]qO}qP(X\x06\x00\x00\x00valuesqQh7X\x08\x00\x00\x00mgr_locsqRcbuiltins\nslice\nqSK\x00K\x05K\x01\x87qTRqUuaustqVbX\x04\x00\x00\x00_typqWX\t\x00\x00\x00dataframeqXX\t\x00\x00\x00_metadataqY]qZX\x05\x00\x00\x00attrsq[}q\\ubh\x03)\x81q]}q^(h\x06h\x07)\x81q_(]q`(h\nh\x0b}qa(h\rh\x0eh\x0fK\x00\x85qbh\x11\x87qcRqd(K\x01K\x05\x85qeh\x18\x89]qf(h\x1cX\r\x00\x00\x00mean_recalledqgX\x11\x00\x00\x00mean_not_recalledqhX\x0c\x00\x00\x00std_recalledqiX\x10\x00\x00\x00std_not_recalledqjetqkbh"Nu\x86qlRqmh\nh\x0b}qn(h\rh\x0eh\x0fK\x00\x85qoh\x11\x87qpRqq(K\x01K\x06\x85qrh\x18\x89]qs(G\x7f\xf8\x00\x00\x00\x00\x00\x00h+h,h-h.h/etqtbh"h1u\x86quRqve]qwh\x0eh\x0fK\x00\x85qxh\x11\x87qyRqz(K\x01K\x05K\x06\x86q{h;\x89C\xf0\x00\x00\x00\x00\x00\x009@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00<@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00 @\xddN\xc5\x08\xc4P\x03\xc0\xd2O8\xb1\xd4\x08\x05\xc0\x11\x0cT\xd0\xa5c\x02\xc0d\xd6 \x94\x1d\xa9\x02\xc0\x9d\xf8\x9bXK\xce\xff\xbf\xb9\xcd\x1f\xcc\xd4\x18\x01\xc0\x14\xdf\xeeof;\x03\xc0\x19\x8eT\xfej\xf2\x04\xc0N)\x8f\xb7g\\\x02\xc0\x84\xb5\xb3\xdf*\x9d\x02\xc0\x99\xddX\x1b\x97\xbf\xff\xbfo\x8e\r&q\x1e\x01\xc0\xafFd\xced\x19\xd5?~\xa4\xc5\xfc=\xcc\xca?\xab)\xaa\xb9\x021\xd9?Y\xe3tp\xab\xdc\xde?\x9e\x92N>%\x8c\xe2?\x94%\xd53k\xf0\xd6?\xd5\x95t\xd0*1\xd5?6\x85\x91\x04\xe5S\xc8?d\xe6\x8fB\xc7X\xd9?UK\x8b>\xc6\xee\xde?W\x9c@\xa8\x12}\xe2?\xc5\xe2E\xacIK\xd7?q|tq}ba]q~h\nh\x0b}q\x7f(h\rh\x0eh\x0fK\x00\x85q\x80h\x11\x87q\x81Rq\x82(K\x01K\x05\x85q\x83h\x18\x89]q\x84(h\x1chghhhihjetq\x85bh"Nu\x86q\x86Rq\x87a}q\x88hK}q\x89(hMh`hN]q\x8a}q\x8b(hQhzhRhSK\x00K\x05K\x01\x87q\x8cRq\x8duaustq\x8ebhWhXhYhZh[}q\x8fubes.'


---

**R1377M** - 3


In [ ]:
byte_alpha = b'\x80\x03}q\x00X\x06\x00\x00\x00R1377Mq\x01]q\x02(cpandas.core.frame\nDataFrame\nq\x03)\x81q\x04}q\x05(X\x05\x00\x00\x00_dataq\x06cpandas.core.internals.managers\nBlockManager\nq\x07)\x81q\x08(]q\t(cpandas.core.indexes.base\n_new_Index\nq\ncpandas.core.indexes.base\nIndex\nq\x0b}q\x0c(X\x04\x00\x00\x00dataq\rcnumpy.core.multiarray\n_reconstruct\nq\x0ecnumpy\nndarray\nq\x0fK\x00\x85q\x10C\x01bq\x11\x87q\x12Rq\x13(K\x01K\x05\x85q\x14cnumpy\ndtype\nq\x15X\x02\x00\x00\x00O8q\x16K\x00K\x01\x87q\x17Rq\x18(K\x03X\x01\x00\x00\x00|q\x19NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?tq\x1ab\x89]q\x1b(X\x05\x00\x00\x00countq\x1cX\r\x00\x00\x00mean_recalledq\x1dX\x11\x00\x00\x00mean_not_recalledq\x1eX\x0c\x00\x00\x00std_recalledq\x1fX\x10\x00\x00\x00std_not_recalledq etq!bX\x04\x00\x00\x00nameq"Nu\x86q#Rq$h\nh\x0b}q%(h\rh\x0eh\x0fK\x00\x85q&h\x11\x87q\'Rq((K\x01K\x14\x85q)h\x18\x89]q*(X\x06\x00\x00\x00insulaq+X\x08\x00\x00\x00fusiformq,X\x10\x00\x00\x00inferiortemporalq-X\x0e\x00\x00\x00middletemporalq.G\x7f\xf8\x00\x00\x00\x00\x00\x00X\x07\x00\x00\x00unknownq/X\x17\x00\x00\x00caudalanteriorcingulateq0X\x0f\x00\x00\x00superiorfrontalq1X\x13\x00\x00\x00caudalmiddlefrontalq2X\x14\x00\x00\x00rostralmiddlefrontalq3X\x10\x00\x00\x00parstriangularisq4X\x0f\x00\x00\x00parsopercularisq5X\x12\x00\x00\x00posteriorcingulateq6X\n\x00\x00\x00precentralq7X\x0b\x00\x00\x00postcentralq8X\r\x00\x00\x00supramarginalq9X\x08\x00\x00\x00banksstsq:X\x07\x00\x00\x00lingualq;X\t\x00\x00\x00precuneusq<X\x10\x00\x00\x00inferiorparietalq=etq>bh"X\x04\x00\x00\x00areaq?u\x86q@RqAe]qBh\x0eh\x0fK\x00\x85qCh\x11\x87qDRqE(K\x01K\x05K\x14\x86qFh\x15X\x02\x00\x00\x00f8qGK\x00K\x01\x87qHRqI(K\x03X\x01\x00\x00\x00<qJNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tqKb\x89B \x03\x00\x00\x00\x00\x00\x00\x00\x00<@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x001@\x00\x00\x00\x00\x00\x00$@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00&@\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x10@\xf6\xd1c\xf8\x0b\xd4\x06\xc0\xddJ\xe2\xea\xae\xf1\x07\xc0\xf8\xf8\xd8\xb9\xda\x96\x07\xc0A\xee4,\xba\x0c\x08\xc0\xa38\x00\xadLy\x07\xc0gh\x1a\xa6\xc3:\x07\xc0\x9e\xf5\x99\x02]^\x07\xc0\xfc\xde\x12\xe3wE\x06\xc0\xbb\x94\xd3\x11_@\x06\xc02fw\xef\xec"\x06\xc0\x11\xd0DiE\xa1\x04\xc0\xd6C\x9b\xcehJ\x07\xc0B\xd4\xc9a\xea\\\x07\xc07\xf3\x15\xe7\x1f$\x07\xc0y/\x8cS\xce\xee\x07\xc0\xb2\x14\x08\xd3S9\x06\xc0J\xaa/\xa6\xa98\x06\xc0\xc0\xf3\x0e\x17\xf7\xca\x06\xc0\xa5\xb4\xa0\x8a\xf6\x14\x08\xc0\x10\xc8#\\J\x8d\t\xc0\x7f\xd8J\xa2t\xd1\x06\xc0\xcd%\t\xc6x\xe2\x07\xc0\x99f\n\xf9\x0e\x98\x07\xc0\'\xe6\xad\xc4\x8a\x14\x08\xc0\xc5\xd1\x03\xb0\x9et\x07\xc0\x1d\x03\xf4\xfd\x957\x07\xc0e\xa8\xdcgsQ\x07\xc0\x85\xcaP\rS3\x06\xc0\xa3\xc3\xbb\x0f)B\x06\xc0\xc3e3\xdb!q\x06\xc0\x88J\xa5\xaax\xe1\x04\xc0(f\xda\xc7\xea\x9f\x07\xc0\x15z8\xecE0\x07\xc0\x1a\xf3\x1bioF\x07\xc0\xc0q\xfa\xf7]\x0c\x08\xc02^z\x06%p\x06\xc0\x1b\x84\xee\x8d\xc0\x92\x06\xc0k\x88\xc9\x98T\xe0\x06\xc0\x93\xa5T\x04\xfa\x17\x08\xc0\xecz\xd1\x8e\xc8\xa1\t\xc03\xe6\x9b!\x0c\xe3\xce?}\xcdOh\xcc\x08\xc9?\xdd\xd3\xc1"\x1f0\xa3?\xfb%zB\x81\x95\xcb?\xbd\xf6\x00\xe5\x8aY\xc7?\x00\x00\x00\x00\x00\x00\x00\x00H\x9e\xcfO\xfb7\xce?\xcc=:0>\x05\xca?\x1eu\x14\x84D\t\xcd?\x80\x90\x88\xb1v\xcc\x9b?\x00\x00\x00\x00\x00\x00\x00\x00\x80\xdc\x13\x88\xbbk\x81?\xf3G\xd7\xe1\xb4\x95\xbf?\x12\xdf\xca\xaa\x9c\xe8\xd2?\xb3\xfeo3c\xb5\xcc?l/\xa3\x13B!\xd1?L,\xdc+\x80\x1a\xc2?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08Gx\xd9\xab\xcc\xa7?\xd7\xc4\xe3\xda\x99_\xd0?\x9e\xc7\xf3\xbcOO\xcb?\xdd\x81_\xd0\x13\xcc\xa3?W\x9a^\xadz\xd7\xcb?&x\xa1\xef\xe7h\xc7?\x00\x00\x00\x00\x00\x00\x00\x00\x12c\xc9t\xc9\xce\xcf?V\x05\\oC\xe9\xc8?.\x14\x01\xfa\x16\x19\xcd?\x80\xa6T0\x9f\x11\x91?\x00\x00\x00\x00\x00\x00\x00\x00\x80\xc7\x1c\xffK\x9b\x8b?1E\xf2\xc9\x0c:\xc0?\x01\x0ebz\xc0\x03\xd1?/\x882\xf5\x89\x03\xcd?\x82\x1d\xbf}od\xd1?t\x8f\xc3\x8f\xa1m\xc1?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa7\xae\xda\x1cl\xad\xa7?qLtqMba]qNh\nh\x0b}qO(h\rh\x0eh\x0fK\x00\x85qPh\x11\x87qQRqR(K\x01K\x05\x85qSh\x18\x89]qT(h\x1ch\x1dh\x1eh\x1fh etqUbh"Nu\x86qVRqWa}qXX\x06\x00\x00\x000.14.1qY}qZ(X\x04\x00\x00\x00axesq[h\tX\x06\x00\x00\x00blocksq\\]q]}q^(X\x06\x00\x00\x00valuesq_hEX\x08\x00\x00\x00mgr_locsq`cbuiltins\nslice\nqaK\x00K\x05K\x01\x87qbRqcuaustqdbX\x04\x00\x00\x00_typqeX\t\x00\x00\x00dataframeqfX\t\x00\x00\x00_metadataqg]qhX\x05\x00\x00\x00attrsqi}qjubh\x03)\x81qk}ql(h\x06h\x07)\x81qm(]qn(h\nh\x0b}qo(h\rh\x0eh\x0fK\x00\x85qph\x11\x87qqRqr(K\x01K\x05\x85qsh\x18\x89]qt(h\x1cX\r\x00\x00\x00mean_recalledquX\x11\x00\x00\x00mean_not_recalledqvX\x0c\x00\x00\x00std_recalledqwX\x10\x00\x00\x00std_not_recalledqxetqybh"Nu\x86qzRq{h\nh\x0b}q|(h\rh\x0eh\x0fK\x00\x85q}h\x11\x87q~Rq\x7f(K\x01K\x14\x85q\x80h\x18\x89]q\x81(h+h,h-h.G\x7f\xf8\x00\x00\x00\x00\x00\x00h/h0h1h2h3h4h5h6h7h8h9h:h;h<h=etq\x82bh"h?u\x86q\x83Rq\x84e]q\x85h\x0eh\x0fK\x00\x85q\x86h\x11\x87q\x87Rq\x88(K\x01K\x05K\x14\x86q\x89hI\x89B \x03\x00\x00\x00\x00\x00\x00\x00\x00<@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x001@\x00\x00\x00\x00\x00\x00$@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00&@\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x10@\xfboa\x85\x07\xec\x06\xc0=^\xf7\x1fLB\x06\xc0\xa19\xdd\xffk\xdc\x05\xc0\xb7\x86\xcc\xb0;\xf7\x06\xc0\x8a[\xbb}d\x8d\x07\xc0\r-\xc9?t\xe0\x06\xc04r\n\xc1\x14\x9b\x06\xc0\x14\xa7AQ\xb1\xa7\x07\xc0\xcf\x861r\xb2c\x06\xc0\x03.\x82I\xa5K\t\xc0T\x89\x04\x895\x87\x06\xc04\x8aq\x1c\x98\x15\x08\xc0,\xd32\xa9\xf7\xfe\x06\xc0\xe4\x08\x9cA\x9d\xbf\x06\xc0\x81\x1aL\xa8O\x82\t\xc0}Wq"K~\x07\xc0>\x1aEC \xa6\x05\xc0k\x08\x99a\xb7\xaf\x05\xc0;\xa3!C\xc89\x07\xc0.\x80\xb9H\x00f\x08\xc0\x8d\\\x0c\xee3\xeb\x06\xc0Cm\xde\xa3|F\x06\xc0\xcb\xd0\x99\xd2%\xe1\x05\xc0\tEQM\x11\xf4\x06\xc0\xea\x9d\xa05\xb5\x8a\x07\xc0\xd7o6\x11\xe5\xd9\x06\xc0r\x15\\\xa8"\x98\x06\xc0\xf0TY\x0b\xff\xa9\x07\xc0j\xb9\x10x@g\x06\xc0-\x81\x98U2?\t\xc0a\xd1)W\xb2\x91\x06\xc0T+\x9cJt\x1c\x08\xc0a\xb2;c\xc2\xfa\x06\xc0\x10\x86\x9a\xfd\x16\xb5\x06\xc0\x04\x8dq"Dz\t\xc0E[\x8a\xd8]y\x07\xc0\xc6\xa8@\x1a\xac\xb4\x05\xc0\xfdG\x1a\xe27\xb1\x05\xc0\x0c;&\xb3\xc9@\x07\xc0\xbcK\xe4n\xeb_\x08\xc0\x90)\\\xe9\x91\xd6\xcc?\x87\xfb\xda,2G\xd2?\xf0\x01\x81\xab$\xdb\xb4?\xc3\x92\xecm\xa2\xe2\xcd?3h\'\xf6\xed\xc9\xc8?\x00\x00\x00\x00\x00\x00\x00\x00\xea\xd5\xbett\xdc\xd1?\xe2\r\xe7\xb7q\xf7\xd1?Q\xfa\x82\xcaU\xe3\xcd? A\xee\x1e\xb8\xd4\xb2?\x00\x00\x00\x00\x00\x00\x00\x00\x00\xae\xd0D\r\xecl?!\xe4]\x00\xa3\x99\xc6?\xb4\xb6Ryl=\xdc?\xe5\xf9`kAq\xae?\x80\xc8\x80\xebZs\xc4?\xfa\xa0\xf3\xc3\x01\xb5\xcd?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc5\xd1y\xa9\xe0\x05\xb4?`\xb4\xc8\x0bo\xf9\xcc?w^y\x89\xe7\x7f\xd2?\x15\xef\x90g2\xb4\xb4?\xd2\xa16\x03\xc4\xb6\xcd?^\xb7K\xbb\xd3\xf9\xc8?\x00\x00\x00\x00\x00\x00\x00\x00\xe6\xb2o\x00\xcc\xaa\xd1?\xf8Su)\xc8\xfa\xd1?\x9bl\xcb@\xe6l\xcd?@\xa0\x87\x02\xbc\x8a\xb1?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x88/\xee\x98\x14M?\xee\x82\xc3\xdb\xd3[\xc6?\xd7\xe8\xe6F\xc8X\xdc?p\xa4\xaf\x84\x93\x9f\xad?7\xa6\x0c\x93|\x12\xc4?\xdc\xb2\xdaz\xa0\xb7\xcd?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00?\xc2\xe2\xd7\xb1J\xb3?q\x8atq\x8bba]q\x8ch\nh\x0b}q\x8d(h\rh\x0eh\x0fK\x00\x85q\x8eh\x11\x87q\x8fRq\x90(K\x01K\x05\x85q\x91h\x18\x89]q\x92(h\x1chuhvhwhxetq\x93bh"Nu\x86q\x94Rq\x95a}q\x96hY}q\x97(h[hnh\\]q\x98}q\x99(h_h\x88h`haK\x00K\x05K\x01\x87q\x9aRq\x9buaustq\x9cbhehfhghhhi}q\x9dubh\x03)\x81q\x9e}q\x9f(h\x06h\x07)\x81q\xa0(]q\xa1(h\nh\x0b}q\xa2(h\rh\x0eh\x0fK\x00\x85q\xa3h\x11\x87q\xa4Rq\xa5(K\x01K\x05\x85q\xa6h\x18\x89]q\xa7(h\x1cX\r\x00\x00\x00mean_recalledq\xa8X\x11\x00\x00\x00mean_not_recalledq\xa9X\x0c\x00\x00\x00std_recalledq\xaaX\x10\x00\x00\x00std_not_recalledq\xabetq\xacbh"Nu\x86q\xadRq\xaeh\nh\x0b}q\xaf(h\rh\x0eh\x0fK\x00\x85q\xb0h\x11\x87q\xb1Rq\xb2(K\x01K\x14\x85q\xb3h\x18\x89]q\xb4(h+h,h-h.G\x7f\xf8\x00\x00\x00\x00\x00\x00h/h0h1h2h3h4h5h6h7h8h9h:h;h<h=etq\xb5bh"h?u\x86q\xb6Rq\xb7e]q\xb8h\x0eh\x0fK\x00\x85q\xb9h\x11\x87q\xbaRq\xbb(K\x01K\x05K\x14\x86q\xbchI\x89B \x03\x00\x00\x00\x00\x00\x00\x00\x00<@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x001@\x00\x00\x00\x00\x00\x00$@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00&@\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x10@\x87\xd9\xbed\xd5\x9a\x06\xc0\x8d?\x7f\xc1x<\x07\xc0p\r\x99\t3\x84\x07\xc0\xf3T\x14R\x91"\x08\xc0\xda\xc7\x14\xd3\x92\xda\x06\xc0\xbcg\xc4\x0e\xd7\x88\x07\xc0\\_o<\xee|\x06\xc0\x96=\xc2\xe8H\xe8\x07\xc0\x96\xa3D\xfbA/\x06\xc0\x00\x1e\xac\xe0t\x95\x06\xc0\n+j\xbd\x02=\x05\xc0-\xa0P\x1dX\xd5\x07\xc0J\xf0\xc3\x9c\x90O\x06\xc0\x92\xdbu\x9bL\xb3\x07\xc0xB\xac\xff>\x84\t\xc0B\x03\xd0\xac~\x86\x07\xc0\x8b\x12+\xa2\x18\xc0\x05\xc0\xd60A\x90\xf7\xb4\x05\xc0WPR1\xc45\x07\xc02\'7\xb7Gs\x08\xc0\xf8\xc0-\xb4\x9d\xa6\x06\xc0(*\xd7\xd7\x83F\x07\xc0\x01\x10\x03\xec\x8f\x8e\x07\xc0Z\x1e_+\x0f*\x08\xc0LMK\x127\xe0\x06\xc0\x0b\xf3\xd4F\x08\x7f\x07\xc0\xb4\x065\\\xcd\x85\x06\xc0\xc7\xda\xf2\xe03\xe4\x07\xc0(2\xf3\xe2\x7fC\x06\xc0\xba\xdby\x14h\xcb\x06\xc0\xb3vK\xad\xfcF\x05\xc0\xda\xa5a\xa1\xb2\xe0\x07\xc0\xe2\xa3\xd4)PA\x06\xc0\x18\xab\xe1=y\xba\x07\xc0<Ll\xe1\xa6\x83\t\xc0u\xff\xfcyF\x88\x07\xc0m\xcb\xdf3\x1e\xc5\x05\xc0\xf3\x8c^\x85;\x9d\x05\xc0\xe2\xe7\xd7:\x02;\x07\xc0\x13\x8e\xc3\x8b\xa2u\x08\xc0~\x96\xa8\xf2}\xb5\xd0?$\x86G\xd3\xc7\xb4\xc7?C\xd8&T\x0c\xf7\xa7?nk\xe9\xb1\xf8\x94\xcd?+rg\xdbd\xf5\xcb?\x00\x00\x00\x00\x00\x00\x00\x00\x9d\xfd\xbf\xfa\x14\xc5\xd1?\ts\x9bN\x85N\xcf?.=\x08\xe9?\x84\xc7?\xc0\xab\xa9H\xdc8\x9a?\x00\x00\x00\x00\x00\x00\x00\x00\x00l0\x0cJ\xe1{?s1}T\r\xb9\xc4?\xe3z+\x00(n\xd4?\xe6\xab\x9aA\x90\xef\xaa?\xbb\x99P~\xf8\xf1\xc3?U\xbc|\x92\xde\xde\xca?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00b\xeaS\x1c\x13|\xae?\x87\xd3i\x05\x07\x8d\xd0?U,J\x94\x08\xe8\xc7?\x80,\x93\xd9\xe5\xd4\xa2?\xbc\xf8<\x97\x8f\xc9\xcd?\xfc\x94a\xd2;8\xcb?\x00\x00\x00\x00\x00\x00\x00\x00"\x7fm\x1b&\xd0\xd1?ZK\x179$\xaf\xcf?\xb5\xe5\xa0l\xff\'\xc6?\x00\x10\x96\xc0Kq\x95?\x00\x00\x00\x00\x00\x00\x00\x00\xc0\xb0\x8e\xa6:\r\x9d?\xdc\xffK\x97\xf3 \xc5?,%\xd8\x83\x89\xb5\xd3?`\xaa.\xa1\xe5\x8d\xab?\xcb(\xb8@|\xe2\xc3?B\xe2\x86\xf0\x1bK\xcb?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x99\xd1\x19\xf6\xb6\xaf\xae?q\xbdtq\xbeba]q\xbfh\nh\x0b}q\xc0(h\rh\x0eh\x0fK\x00\x85q\xc1h\x11\x87q\xc2Rq\xc3(K\x01K\x05\x85q\xc4h\x18\x89]q\xc5(h\x1ch\xa8h\xa9h\xaah\xabetq\xc6bh"Nu\x86q\xc7Rq\xc8a}q\xc9hY}q\xca(h[h\xa1h\\]q\xcb}q\xcc(h_h\xbbh`haK\x00K\x05K\x01\x87q\xcdRq\xceuaustq\xcfbhehfhghhhi}q\xd0ubh\x03)\x81q\xd1}q\xd2(h\x06h\x07)\x81q\xd3(]q\xd4(h\nh\x0b}q\xd5(h\rh\x0eh\x0fK\x00\x85q\xd6h\x11\x87q\xd7Rq\xd8(K\x01K\x05\x85q\xd9h\x18\x89]q\xda(h\x1cX\r\x00\x00\x00mean_recalledq\xdbX\x11\x00\x00\x00mean_not_recalledq\xdcX\x0c\x00\x00\x00std_recalledq\xddX\x10\x00\x00\x00std_not_recalledq\xdeetq\xdfbh"Nu\x86q\xe0Rq\xe1h\nh\x0b}q\xe2(h\rh\x0eh\x0fK\x00\x85q\xe3h\x11\x87q\xe4Rq\xe5(K\x01K\x14\x85q\xe6h\x18\x89]q\xe7(h+h,h-h.G\x7f\xf8\x00\x00\x00\x00\x00\x00h/h0h1h2h3h4h5h6h7h8h9h:h;h<h=etq\xe8bh"h?u\x86q\xe9Rq\xeae]q\xebh\x0eh\x0fK\x00\x85q\xech\x11\x87q\xedRq\xee(K\x01K\x05K\x14\x86q\xefhI\x89B \x03\x00\x00\x00\x00\x00\x00\x00\x00<@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x001@\x00\x00\x00\x00\x00\x00$@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00&@\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x10@e1J\xf6\xd0m\x06\xc0P\x13\x80\x87\xabA\x07\xc0\xc8\xf5m\x04\xdac\x07\xc0\x1e\xfa\xba\xc1m\xf3\x07\xc0^\xbb\x97\xbb\xf7\xd9\x06\xc02O\x9c\x12\xf5J\x07\xc0\xcd\xdd\x02\xec\xe6j\x06\xc0\xc6\x9dl\x08\xaa\xc1\x07\xc0\x12M\x85C)!\x06\xc0Hk\xf2\x06\x07v\x06\xc0`\xb8\x08\x92\xf4\xe3\x04\xc0\xc0{\x10\xb2\xf2\xc5\x07\xc05M\xeax\x932\x07\xc0\xe5\x82\xf3"\x1am\x07\xc0=\x11)\xa0\xdfl\t\xc0~\xeeB\xe1\xe4w\x07\xc0\x90\xf2\xc3#\xb9\xa2\x05\xc0j\xe6\x81K\x8e\xcf\x05\xc0\x80\xd7\xba\x84p8\x07\xc0=)\xc7%8Y\x08\xc0\xf5\x08H\xd7\x00b\x06\xc0sC\x8aV\x8f7\x07\xc0\xe1\x84\xa7\xb6\x80w\x07\xc0%Q`k\x87\x12\x08\xc0\x8f\'M6!\xc2\x06\xc0\x0f\x07\xae\xa4aI\x07\xc0\x95\xe1\x8b\x10^v\x06\xc0\xaa\xb3<\xbe\xf6\xc2\x07\xc0\xa1\xae\x01\xa2-\x0e\x06\xc0r}>\xd0\xb2\xa1\x06\xc0\xc2\x8b\x1a\x8b\x876\x05\xc0\xa9\xdd\xfe\xc0!\xf4\x07\xc0\xfeF\x8a\x8d\x11\x0f\x07\xc0Y\x8b\xcc\xd2\xfbg\x07\xc0\xe0\xb1\xad\xc9:k\t\xc0\x18\x12T\x161~\x07\xc0d\xb1\xb6\xaes\x97\x05\xc0\xda\xb1\x10\x8d\xe4\xcf\x05\xc0T\xaf\x9ct\x01>\x07\xc0\xb6\x0f\xce\xcc4V\x08\xc0\xa6\xbc\xb9T\t5\xd1?\n\x9e\x1e7\x05"\xc8?\x1fc\x00\xf6\xd3X\xa5?\\\x83)42\x84\xce?\xfe{\x0f\x15\x84T\xca?\x00\x00\x00\x00\x00\x00\x00\x00\xbf>\x1d\xd4\xf7\x0f\xd2?\xbc\xe5\xa3\xb8l\x9c\xd0?$_~f\x08\xd6\xc5?\x00\xe9\xa2\t\x89\xb9s?\x00\x00\x00\x00\x00\x00\x00\x00`W\xbc\xf1[w\xa0?\x85>$\xdb\xe7W\xc8?\xb9\xb3/\xde\xf8\x9a\xd1?-\xc7|;F\x87\xac?\xd4\x9d\xec\xf6\x88A\xc5?_\x15\x1fh\xbfW\xcd?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00G9\x84\xf2n\xc1\xb3?/\xa9\x89\n\x83\x90\xd1?0\x9d.\x98wH\xc9?\xfd\x05\x80E\xc7\xe3\xa5?\x9d\xd2_\x90\xb4\xee\xce?\x07\x19\xdf\x92)\x8f\xcc?\x00\x00\x00\x00\x00\x00\x00\x00X\xbdO\x11\x00\xaa\xd1?l\x9e\x07\t\xebj\xd0?p\x81\x8a\xc9\x0b\x14\xc7?\x00\x962\xc1\x9f\\o?\x00\x00\x00\x00\x00\x00\x00\x00\xc0\x11P\xfe5\xe4\xa6?\x9f)\x02\xb6\xf4\xee\xc8?\x12$\xfa\xc3\xc5\x1c\xd2?v\xd5b\xac\xa3U\xab?x\xc3\xb2O\xfd\xa5\xc5?\xa8wsL@2\xcd?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00B\xc8{<\x10\x9e\xb3?q\xf0tq\xf1ba]q\xf2h\nh\x0b}q\xf3(h\rh\x0eh\x0fK\x00\x85q\xf4h\x11\x87q\xf5Rq\xf6(K\x01K\x05\x85q\xf7h\x18\x89]q\xf8(h\x1ch\xdbh\xdch\xddh\xdeetq\xf9bh"Nu\x86q\xfaRq\xfba}q\xfchY}q\xfd(h[h\xd4h\\]q\xfe}q\xff(h_h\xeeh`haK\x00K\x05K\x01\x87r\x00\x01\x00\x00Rr\x01\x01\x00\x00uaustr\x02\x01\x00\x00bhehfhghhhi}r\x03\x01\x00\x00ubes.'

---

**R1065J** - 4


In [ ]:
byte_alpha = b'\x80\x03}q\x00X\x06\x00\x00\x00R1065Jq\x01]q\x02(cpandas.core.frame\nDataFrame\nq\x03)\x81q\x04}q\x05(X\x05\x00\x00\x00_dataq\x06cpandas.core.internals.managers\nBlockManager\nq\x07)\x81q\x08(]q\t(cpandas.core.indexes.base\n_new_Index\nq\ncpandas.core.indexes.base\nIndex\nq\x0b}q\x0c(X\x04\x00\x00\x00dataq\rcnumpy.core.multiarray\n_reconstruct\nq\x0ecnumpy\nndarray\nq\x0fK\x00\x85q\x10C\x01bq\x11\x87q\x12Rq\x13(K\x01K\x05\x85q\x14cnumpy\ndtype\nq\x15X\x02\x00\x00\x00O8q\x16K\x00K\x01\x87q\x17Rq\x18(K\x03X\x01\x00\x00\x00|q\x19NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?tq\x1ab\x89]q\x1b(X\x05\x00\x00\x00countq\x1cX\r\x00\x00\x00mean_recalledq\x1dX\x11\x00\x00\x00mean_not_recalledq\x1eX\x0c\x00\x00\x00std_recalledq\x1fX\x10\x00\x00\x00std_not_recalledq etq!bX\x04\x00\x00\x00nameq"Nu\x86q#Rq$h\nh\x0b}q%(h\rh\x0eh\x0fK\x00\x85q&h\x11\x87q\'Rq((K\x01K\x12\x85q)h\x18\x89]q*(X\t\x00\x00\x00precuneusq+X\x10\x00\x00\x00superiorparietalq,X\x10\x00\x00\x00inferiorparietalq-X\r\x00\x00\x00supramarginalq.G\x7f\xf8\x00\x00\x00\x00\x00\x00X\x0f\x00\x00\x00parahippocampalq/X\x08\x00\x00\x00fusiformq0X\x10\x00\x00\x00inferiortemporalq1X\x10\x00\x00\x00superiortemporalq2X\x0e\x00\x00\x00middletemporalq3X\x13\x00\x00\x00medialorbitofrontalq4X\x14\x00\x00\x00lateralorbitofrontalq5X\x10\x00\x00\x00parstriangularisq6X\r\x00\x00\x00parsorbitalisq7X\x0f\x00\x00\x00parsopercularisq8X\x06\x00\x00\x00insulaq9X\x12\x00\x00\x00transversetemporalq:X\x14\x00\x00\x00rostralmiddlefrontalq;etq<bh"X\x04\x00\x00\x00areaq=u\x86q>Rq?e]q@h\x0eh\x0fK\x00\x85qAh\x11\x87qBRqC(K\x01K\x05K\x12\x86qDh\x15X\x02\x00\x00\x00f8qEK\x00K\x01\x87qFRqG(K\x03X\x01\x00\x00\x00<qHNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tqIb\x89B\xd0\x02\x00\x00\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x007@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00.@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00.@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00F@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00*@\x99\xabo\xcf\xfb\xbf\x03\xc0\x10\x04\x87\xc9\xb8\xa1\x08\xc0\x90""\x07\x16\xb4\x04\xc0\x12\x0b\x12\xd9\x0c\xf4\x03\xc0T\xfbD#\xb5\xa0\x03\xc0^\x7f:a\x00D\x03\xc0\xdc\xc7\x9e8\xd7\x8f\x05\xc0\x84\xd5hh\xf3f\x06\xc0_\xb1V\x0c]\n\x05\xc0\x8b\xa3\x05\x07\x88\x84\x06\xc0\xc5!\xa6H\x05\xaf\x01\xc0QYd\xc5\xde\x87\x03\xc0\x15\x8dc\x93\xd6b\x03\xc0@\xdeC\x80\xfbd\x03\xc0\xcf^\x04\xca\xed?\xfd\xbf\xd9\x1c\x07Z\xc3%\x02\xc0\xa6\xe3\xd8\x9dd)\x00\xc0;\xa3x\xc5\xa3\x96\x03\xc0\x16\xebI\x11\x07\xe7\x03\xc0~\xbd\xc6\xd4\xcc\xb2\x08\xc0(\xf9\x92\xf26\xc6\x04\xc0\xb4\xbf\x9b%\xbf\x18\x04\xc0\xe2jnw\xe8\xfa\x03\xc0\x96$+\xa2\xe9\xf8\x03\xc0\x89\xc2\x9a\x03\xdc\xfd\x05\xc0\x12\xf3\xc8\xb2\x14\xad\x06\xc0\x17\xc9\xfa\xe7\x17,\x05\xc0d\xf5r+N\x05\x07\xc0D\xf6\x93F\xd6\xe0\x01\xc07\r\xf6\x11\x9b\x96\x03\xc0\'T\tn=3\x03\xc0\x96\xcdY\x8e\xe5\xa4\x03\xc0\rO\xda0\xee\xa1\xfd\xbf\xde\xee>\xc1K&\x02\xc0\x11\xb1\x9bT\x1f/\x00\xc0\x10V\xc8\x15\x14\x8c\x03\xc0\xcd\xc3.\xf6\r+\xd2?1m\xa3b:\xcb\xce?Q\x83K\xea\x18K\xcc?\r\x92^L\x16I\xde?\xc4\x1b\x94:r\xa4\xd3?\x00=,\x08\xf1tv?\x8a\xba\xa0\x99\xf8\xe6\xd2?\x83E\xc0\x03\xd9\x85\xc3?\xc8\xd6.\x10o\x9b\xdc?KZ\xf4\xe7\xe1\xb0\xd5?\x00\x00\x00\x00\x00\x00\x00\x00C\x91\x82nq%\xd4?$\x86\xba\x92\x02y\xcd?\x10\xcfa\xeexX\xb0?\x00\x00\x00\x00\x00\x00\x00\x00\x9a\x1c+9w0\xd4?06\x88\xb1#J\xbd?.\x0e\x8a\xc8=\xf8\xd0?U~e\xa2\xc0\xdc\xd2?g.\xd3y\xbc"\xcd?\xda\x1e(6\xdae\xce?\xed\xb6t\x16\xc6\xde\xdd?:\xdb\xc6\x99\t1\xd4?\x00^\x12\xdb\x80\x02d?\xa8s\x83\xf3\xc8\xf2\xcf?\xfd\x8e\xac\xad\xf8\xd7\xc3?\xa8\xf9_\xc9{\xf4\xdb?v\xc8\xef\xe4Y\xd1\xd4?\x00\x00\x00\x00\x00\x00\x00\x00\xab\x04\x82\xffQ\xcc\xd3?~\xf1\xc1\x9d\xcdM\xcf?1\x8eXo\xdbm\xb7?\x00\x00\x00\x00\x00\x00\x00\x00\x14\x1et\xd6\x05\x84\xd4?@\xde5N\x94\x93\xb7?\t\xd3\x7f\x085p\xd1?qJtqKba]qLh\nh\x0b}qM(h\rh\x0eh\x0fK\x00\x85qNh\x11\x87qORqP(K\x01K\x05\x85qQh\x18\x89]qR(h\x1ch\x1dh\x1eh\x1fh etqSbh"Nu\x86qTRqUa}qVX\x06\x00\x00\x000.14.1qW}qX(X\x04\x00\x00\x00axesqYh\tX\x06\x00\x00\x00blocksqZ]q[}q\\(X\x06\x00\x00\x00valuesq]hCX\x08\x00\x00\x00mgr_locsq^cbuiltins\nslice\nq_K\x00K\x05K\x01\x87q`RqauaustqbbX\x04\x00\x00\x00_typqcX\t\x00\x00\x00dataframeqdX\t\x00\x00\x00_metadataqe]qfX\x05\x00\x00\x00attrsqg}qhubh\x03)\x81qi}qj(h\x06h\x07)\x81qk(]ql(h\nh\x0b}qm(h\rh\x0eh\x0fK\x00\x85qnh\x11\x87qoRqp(K\x01K\x05\x85qqh\x18\x89]qr(h\x1cX\r\x00\x00\x00mean_recalledqsX\x11\x00\x00\x00mean_not_recalledqtX\x0c\x00\x00\x00std_recalledquX\x10\x00\x00\x00std_not_recalledqvetqwbh"Nu\x86qxRqyh\nh\x0b}qz(h\rh\x0eh\x0fK\x00\x85q{h\x11\x87q|Rq}(K\x01K\x12\x85q~h\x18\x89]q\x7f(h+h,h-h.G\x7f\xf8\x00\x00\x00\x00\x00\x00h/h0h1h2h3h4h5h6h7h8h9h:h;etq\x80bh"h=u\x86q\x81Rq\x82e]q\x83h\x0eh\x0fK\x00\x85q\x84h\x11\x87q\x85Rq\x86(K\x01K\x05K\x12\x86q\x87hG\x89B\xd0\x02\x00\x00\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x007@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00.@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00.@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00F@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00*@\xa2-G\x88&\x00\x04\xc0\x19\x92\x99e\x91\xaa\x08\xc0\xaa\xab\xe1\xa3\n-\x05\xc0K\x04"\xf4\x83\xf2\x03\xc0\xe6\xeb<\x1eS$\x04\xc0\x04\xfa\x02\xb6$q\x03\xc0(;\xe3\xed<\x9a\x05\xc0(\x83]\xb1w`\x06\xc0\xa9=\xd4+\xad\x13\x05\xc0\xd5\xdb\x9b\xa5bw\x06\xc0\x08d\xf0\xb1\xd0\xf2\x01\xc0\x12\x17\xbb](\xb4\x03\xc0\x07;\r/Y\xe6\x03\xc0JRB\xdd-\xba\x03\xc0:O\xd7\xd2\x16\xc4\xfd\xbf5 \x10\'!8\x02\xc0a\xb0\xa2\xc4\xa2&\x00\xc0\x96\x97\x0c\xac\xf2\xae\x03\xc0\x10\xc7\xfd\x8fp\x08\x04\xc0\x01\x19\xd4\x9bF\xb2\x08\xc0\xa4\xf4>\xb7!2\x05\xc0%\xcbU\x85o\xe6\x03\xc0\x1f\xe0\x06\xc8\xd7"\x04\xc0\x82\x1fJ$\xaat\x03\xc0\xbf\xa1ro>\x92\x05\xc0>\x07\xc8\xeb\xed\xbf\x06\xc0\xfb6\xda\xdff$\x05\xc0\x91\xfft\xb6W\x90\x06\xc0\x88k\xd5 \xfaL\x02\xc0\x83^S\x87W\xad\x03\xc0\xf8\xc5\xa9\x04\xcb\xbd\x03\xc0vb\x089\x0bp\x03\xc0\x96*\xbd){\x1a\xfe\xbf\x8f\x92C\xdb\x9b-\x02\xc0\x9c\xb1E4\'\xf1\xff\xbf\x11\xca{\xd59\xc0\x03\xc0\x87V[9Y\x01\xd0?\xda=\xa0\xfcK.\xd0?\xe2Gk\'!\x1f\xc8?\x17\x85\xc6cC\xe5\xde?\xcar\xe4\xb76\xf9\xd2?\xc0\xd18\xe0\x948\x9e?\xa5\xe9\xda\xc8\xef:\xd2?\x94N\xbb\'\xf6\xb7\xc4?s\x84\x1f\x05\xc6h\xdc?\xfa\xdc#\xf8qK\xd7?\x00\x00\x00\x00\x00\x00\x00\x00\xd5\xbc\xa9\x9e\x0e\x9d\xd3?\xb9\xa5h\xf6\x19?\xcf?\x90br\xb0kC\xb4?\x00\x00\x00\x00\x00\x00\x00\x00\xc8\xe3\xf1%\xc7\xfa\xd6?\x00G\x11a\xe6\xd0\xba?\xd4I\xd3\xc3\n\xfb\xd0?H\xc9\xf2\xff\xda\x1e\xd0?2\xd1\xc0\xaa\x01\x93\xd0?\xc7\x89^\xb7|Y\xc8? \xb0\x91lT\xae\xde?\xe9\xf3\x95\x0e\xde\xbb\xd2?\xc0\x0eP\xef"\x8c\x94?\xba\xde\xff\xfdR4\xd2?\xa1\n\xe7(\x93d\xc4?\xe4\xd8\x91\xad\xf8\xa5\xdc?\xbb\x7f\xf8\x95\x9f\xa8\xd7?\x00\x00\x00\x00\x00\x00\x00\x00\x1cJ\xa1@\x13\xb5\xd3?\'O\xc1 (|\xd0?\xe0\x95Ht\xc5"\xb5?\x00\x00\x00\x00\x00\x00\x00\x00\xf3\'\xdfB+\x01\xd7?\xa0\xd5\x8d\xe6\xa1\xdb\xbd?rW\xe3\xfc\xec\xdb\xd0?q\x88tq\x89ba]q\x8ah\nh\x0b}q\x8b(h\rh\x0eh\x0fK\x00\x85q\x8ch\x11\x87q\x8dRq\x8e(K\x01K\x05\x85q\x8fh\x18\x89]q\x90(h\x1chshthuhvetq\x91bh"Nu\x86q\x92Rq\x93a}q\x94hW}q\x95(hYhlhZ]q\x96}q\x97(h]h\x86h^h_K\x00K\x05K\x01\x87q\x98Rq\x99uaustq\x9abhchdhehfhg}q\x9bubh\x03)\x81q\x9c}q\x9d(h\x06h\x07)\x81q\x9e(]q\x9f(h\nh\x0b}q\xa0(h\rh\x0eh\x0fK\x00\x85q\xa1h\x11\x87q\xa2Rq\xa3(K\x01K\x05\x85q\xa4h\x18\x89]q\xa5(h\x1cX\r\x00\x00\x00mean_recalledq\xa6X\x11\x00\x00\x00mean_not_recalledq\xa7X\x0c\x00\x00\x00std_recalledq\xa8X\x10\x00\x00\x00std_not_recalledq\xa9etq\xaabh"Nu\x86q\xabRq\xach\nh\x0b}q\xad(h\rh\x0eh\x0fK\x00\x85q\xaeh\x11\x87q\xafRq\xb0(K\x01K\x12\x85q\xb1h\x18\x89]q\xb2(h+h,h-h.G\x7f\xf8\x00\x00\x00\x00\x00\x00h/h0h1h2h3h4h5h6h7h8h9h:h;etq\xb3bh"h=u\x86q\xb4Rq\xb5e]q\xb6h\x0eh\x0fK\x00\x85q\xb7h\x11\x87q\xb8Rq\xb9(K\x01K\x05K\x12\x86q\xbahG\x89B\xd0\x02\x00\x00\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x007@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00.@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00.@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00F@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00*@\xd2\x15\xff\x96\x8ey\x04\xc0\xd0\x8f\xb5w*\xd2\x08\xc0\x7f\x06\x1d92+\x06\xc0+%\xbb\xa4\xe7\x02\x04\xc0\x90\xbd\x806{\xf3\x03\xc0y\xd8\x13\x9f\xe3\xea\x02\xc0\x19\x0c\xd5\xd2._\x05\xc0\x96<\xf3\x9c\\\x06\x06\xc0\x14\xca\x89/1\xdb\x04\xc0!\xab\xc3]3\x0f\x07\xc0\\/\xd1\xcc\r\xcd\x01\xc0\xceLmL\xe8\xb0\x03\xc0\x14\tr\xfb[ \x04\xc0\x80L\x17\x9e\xed\xa1\x03\xc0\x83!\r\xf1v\x85\xfd\xbf\xfb\xb9\xc4h\x16\x01\x02\xc0EmL(\x98A\xff\xbf\x9a\r\xc4\xae\x91\xfa\x03\xc0\xf4\x0697z\xac\x04\xc0.\xf2\x17+\xcc\xf2\x08\xc0<\xfeV\x0e\x9cM\x06\xc0\xa7;\xaf5P \x04\xc0\xde|H6J%\x04\xc0\x1c\xc0XD\xd6&\x03\xc0\xc5\xfeR@\x13\x91\x05\xc0\x98\x85-\xf7\xb2S\x06\xc0\xe1e\xc7{"\xe8\x04\xc0\xb5\xcaK\xc6\x94\x14\x07\xc0\xc2\x12\x9f\xc3\x9c\xfb\x01\xc0\x01\xda\x14\xf2\xd9\xc1\x03\xc0\xbcQg|\x839\x04\xc0\x10\x1a\xdb`\x8c\xad\x03\xc0r\xb6+\xb5-\xc3\xfd\xbf\x17\x04\xfe\x97\xf9\xf3\x01\xc0\xf6\xa0\xfa\xb41p\xff\xbf\x99^\xf29\'\xfa\x03\xc0#\xe7H<\xec\xa0\xcc?\x0fgv\xff\xe6\x1d\xd1?\xe5\xa3>\x03\xecc\xc5?\xff\x9b:\r\xa5#\xdf?\xa0\xcdy>\x95t\xd2?@\x07\x0fI\xcd\xe3\xa6?\xa3\x0b\xfd\xd4}U\xd4?\xc7\xa5\xa3<\x81F\xc3?0\x12\xc6a\xd3i\xdc?(\xc0\xf2\x94\xa2\xb5\xd8?\x00\x00\x00\x00\x00\x00\x00\x00T\x1b\xd6\xba\x16\xf9\xd3?\xf9\x8dZ\xcf\xba,\xd0?p\xa2\xeb\x14m?\xb4?\x00\x00\x00\x00\x00\x00\x00\x00\xa0_fJj\x92\xd3?\xd0\xc8\x16\xe7\x97\xa0\xb9?\xeb:\xe2\x87b\xdb\xcd?\x1a\xbf\xc3\xc1/\xaf\xcb?\xce\xf7\xa1\xd5\xcad\xd0?V\xa5\x96\x10\x97j\xc5?\x93%zo`\xf8\xdd?\xd6:\x01m\x7f\x84\xd2?\x80\x9e\x89/|\xfd\x8b?\xb9<\xc4\r\x89\xbb\xd2?v\xe8{\xb5\xac\x02\xc4?\x80\x05\'g\xcf\x8c\xdd?\xbb3g\xb0U\x03\xd9?\x00\x00\x00\x00\x00\x00\x00\x00Oo\'u\x12\xb9\xd3?v!)\x08F>\xd0?/W8\x8f\x8e\xb6\xb8?\x00\x00\x00\x00\x00\x00\x00\x00\x13\xbb5\x11\x9fQ\xd3?\x18?\xfax\x8cR\xb5?\x90x\xach\x8e\x7f\xcf?q\xbbtq\xbcba]q\xbdh\nh\x0b}q\xbe(h\rh\x0eh\x0fK\x00\x85q\xbfh\x11\x87q\xc0Rq\xc1(K\x01K\x05\x85q\xc2h\x18\x89]q\xc3(h\x1ch\xa6h\xa7h\xa8h\xa9etq\xc4bh"Nu\x86q\xc5Rq\xc6a}q\xc7hW}q\xc8(hYh\x9fhZ]q\xc9}q\xca(h]h\xb9h^h_K\x00K\x05K\x01\x87q\xcbRq\xccuaustq\xcdbhchdhehfhg}q\xceubh\x03)\x81q\xcf}q\xd0(h\x06h\x07)\x81q\xd1(]q\xd2(h\nh\x0b}q\xd3(h\rh\x0eh\x0fK\x00\x85q\xd4h\x11\x87q\xd5Rq\xd6(K\x01K\x05\x85q\xd7h\x18\x89]q\xd8(h\x1cX\r\x00\x00\x00mean_recalledq\xd9X\x11\x00\x00\x00mean_not_recalledq\xdaX\x0c\x00\x00\x00std_recalledq\xdbX\x10\x00\x00\x00std_not_recalledq\xdcetq\xddbh"Nu\x86q\xdeRq\xdfh\nh\x0b}q\xe0(h\rh\x0eh\x0fK\x00\x85q\xe1h\x11\x87q\xe2Rq\xe3(K\x01K\x12\x85q\xe4h\x18\x89]q\xe5(h+h,h-h.G\x7f\xf8\x00\x00\x00\x00\x00\x00h/h0h1h2h3h4h5h6h7h8h9h:h;etq\xe6bh"h=u\x86q\xe7Rq\xe8e]q\xe9h\x0eh\x0fK\x00\x85q\xeah\x11\x87q\xebRq\xec(K\x01K\x05K\x12\x86q\xedhG\x89B\xd0\x02\x00\x00\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x007@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00.@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00.@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00F@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00*@\x03\xe3~g=\xbb\x04\xc0\x9f\xa0\xa5\xa7f\xdc\x08\xc0$\x1aEL\xba\xb8\x06\xc0\x1c\xc3\x8bp\xffD\x04\xc0\xf3\x8c\xc1\xe3\xf4\xed\x03\xc0>9\xeb\x8a\xaem\x02\xc0\x8b6/\xe23Z\x05\xc0\xfa\x92\x90\xbeN\xdb\x06\xc0/_\xb3\xca\x15$\x05\xc0\x0c[9\xd0\xb6\xd4\x06\xc0\x04v\xeeo\xe6\xf3\x01\xc0\xb5n\xe1\xb6\xa7\xbe\x03\xc0-\x904\xf9\rg\x04\xc0&0z\x98\xcc\xaa\x03\xc0\x92\xce\xe0\xc7\x1c\x01\xfe\xbf\xd9hG$th\x02\xc0>\x98\xf73\x96\xe0\xfe\xbf\t\xec\x8b\x00\x0bu\x04\xc0E?\x9c\x84\xbd\xfc\x04\xc0\x8e\x01\xdd\xd4\xda\xf1\x08\xc0d\xa6L\x1c\xbe\xb0\x06\xc0fD\xd1\xda\x90W\x04\xc0DK\x97\x9bg\x19\x04\xc0\x1d\x08\xbc\xf2X\xf8\x02\xc0eJO\x93\x0f\xac\x05\xc0\x0e\xcd\xa2\xdf\xd8 \x07\xc0\xe7\x81\xec\r\xb2/\x05\xc0\x8d\xc7k\xe7\xb2\x7f\x06\xc0\xde\x8dT\x07\x10;\x02\xc0t.\xc4D\xe4\xcd\x03\xc0\xd89\xff\xf3O_\x04\xc0\xba\'\xda-\x8d;\x04\xc0\x04\xae\xfet!n\xfd\xbf/\xec\x9f\x0ePC\x02\xc0\xba\xbah\xb70S\xff\xbf\xd70V!\x93o\x04\xc0\x80<\x1bA\nn\xc7?\xcc\x7f\xaeUr\xfe\xd0?y\xb9l\xdb<\x97\xc7?^Jj\xc2#v\xdb?\x05%\xbd\x08zX\xd3?\x00f\x16T0\x00\x92?X}\xbb\x13\xfc\xbe\xd3?^*\x1aB\x95\xd2\xd2?t\xe5\x82\xde) \xdc?\x83c\xc9\xb6C\xa6\xd9?\x00\x00\x00\x00\x00\x00\x00\x00~dx\x14"\x89\xd3?N q\xdd\xac3\xd1?\x80\x08.bp\xc8\xb1?\x00\x00\x00\x00\x00\x00\x00\x00\x97\\\x1d\xf2d\x13\xd4?\x98;\xefh }\xb5?\x9d\x84\xce\xc1Dh\xcb?.q\ts\xec\x9b\xc8?\xa3?\x96\xf9!E\xd0?\x91\x82\xb0\xf3,\xcf\xc6?\x1d\xee`\x85\xc7~\xdc?\xc1\x07\xf9\xd7\xc9\xb1\xd3?\x00HW\x81@\xc6\x98?\xc2!\x8a\x89\xac\x10\xd3?+b\xc2\x95|\'\xd3?\x8f\xd8\xf9\x9ck%\xdd?\x99\'U\xbe\xcb\xd3\xd8?\x00\x00\x00\x00\x00\x00\x00\x00\xa4Nr\\\x15\xa1\xd3?\x07cel\xee\xf7\xd0?0\x83\xd2Pj\x9b\xba?\x00\x00\x00\x00\x00\x00\x00\x00\xcf\x1e\xa2\x0b?\x8c\xd2?\xb8\xc9,H\xb7:\xbb?\xb5\xc4j\xdd7\x9c\xcc?q\xeetq\xefba]q\xf0h\nh\x0b}q\xf1(h\rh\x0eh\x0fK\x00\x85q\xf2h\x11\x87q\xf3Rq\xf4(K\x01K\x05\x85q\xf5h\x18\x89]q\xf6(h\x1ch\xd9h\xdah\xdbh\xdcetq\xf7bh"Nu\x86q\xf8Rq\xf9a}q\xfahW}q\xfb(hYh\xd2hZ]q\xfc}q\xfd(h]h\xech^h_K\x00K\x05K\x01\x87q\xfeRq\xffuaustr\x00\x01\x00\x00bhchdhehfhg}r\x01\x01\x00\x00ubh\x03)\x81r\x02\x01\x00\x00}r\x03\x01\x00\x00(h\x06h\x07)\x81r\x04\x01\x00\x00(]r\x05\x01\x00\x00(h\nh\x0b}r\x06\x01\x00\x00(h\rh\x0eh\x0fK\x00\x85r\x07\x01\x00\x00h\x11\x87r\x08\x01\x00\x00Rr\t\x01\x00\x00(K\x01K\x05\x85r\n\x01\x00\x00h\x18\x89]r\x0b\x01\x00\x00(h\x1cX\r\x00\x00\x00mean_recalledr\x0c\x01\x00\x00X\x11\x00\x00\x00mean_not_recalledr\r\x01\x00\x00X\x0c\x00\x00\x00std_recalledr\x0e\x01\x00\x00X\x10\x00\x00\x00std_not_recalledr\x0f\x01\x00\x00etr\x10\x01\x00\x00bh"Nu\x86r\x11\x01\x00\x00Rr\x12\x01\x00\x00h\nh\x0b}r\x13\x01\x00\x00(h\rh\x0eh\x0fK\x00\x85r\x14\x01\x00\x00h\x11\x87r\x15\x01\x00\x00Rr\x16\x01\x00\x00(K\x01K\x12\x85r\x17\x01\x00\x00h\x18\x89]r\x18\x01\x00\x00(h+h,h-h.G\x7f\xf8\x00\x00\x00\x00\x00\x00h/h0h1h2h3h4h5h6h7h8h9h:h;etr\x19\x01\x00\x00bh"h=u\x86r\x1a\x01\x00\x00Rr\x1b\x01\x00\x00e]r\x1c\x01\x00\x00h\x0eh\x0fK\x00\x85r\x1d\x01\x00\x00h\x11\x87r\x1e\x01\x00\x00Rr\x1f\x01\x00\x00(K\x01K\x05K\x12\x86r \x01\x00\x00hG\x89B\xd0\x02\x00\x00\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x007@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00.@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00.@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00F@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00*@\xa7\xe4W\xeeY\xd8\x05\xc0Z\xfd\xb0\x98^\xe1\x08\xc03\xdc1_\xc1\xa7\x06\xc0x\x0br\xd0\xd1n\x04\xc0\x8e=\x81^\xb1\x87\x04\xc0\xe0\xc2\n\x18\x9cw\x02\xc00\x16\n\xe1\x1a1\x06\xc0P\x11\xa8\xa4\xdb\x1c\x07\xc0\x8c\x97q\xd7*\x89\x05\xc0k\xdfd\xec\xba/\x07\xc0\xb8\xf2+i\xf5\xad\x02\xc0\x95\r/d=\x98\x04\xc0\xb85\xb6\n\x07\xae\x04\xc0\xb0,\xf2\xcb\x9f\xd9\x04\xc0\x81\x9d\xc0;\xb3\x9c\xfd\xbf\xc1&c\x19WP\x02\xc0<\x00?\xc4U\x8b\xff\xbfE\x00\xafaY\xa0\x05\xc0\xa5\xf5!t\x81\xe1\x05\xc0G1+\xc0-\x0f\t\xc0\xb7\xa3\xba\x11\x0c\xcb\x06\xc04r#O\x7f\x80\x04\xc0`\x8d\x1c\x9a\xa8\xb7\x04\xc0j\xa5\xed\xdbM\x05\x03\xc0k\x85:\xa5\xd9b\x06\xc0\xfd\xd5.\x84\xebw\x07\xc0t\xa9\xdf\x1d\xed\xde\x05\xc0\x89\\_=<{\x07\xc0\x08\\4\x06\xd7\xa1\x02\xc0&\'\xf1 \x11\x92\x04\xc0A\xd3\xed\x9bl\xe6\x04\xc0\xfc\xb9)1a\x0e\x05\xc0\x9c\xf3\x96\xb8c@\xfe\xbf\x86\xb1\xfaBwR\x02\xc0\xd8Y\xf2\x90f\x88\xff\xbf\x84\x07_\xa0I\xa5\x05\xc0\xdb\xe1#\x93F<\xc0?\'v\xa0\xa0E\\\xcc?\x8d\xdb\xda\xf5\x01\xed\xc6?\'\xedF\x0b5\xfa\xdc?\xe9\x1c\x0f<\x83\xfb\xd5?\xc0\xaaX>\x9aO\x94?\x81\xf1`>F\x13\xd6?D\xbd\x07\xb9\xde4\xc8?\x7f\x8d\x81\xf8\xe8\x11\xde?\xe6d\x0c\xc3B"\xda?\x00\x00\x00\x00\x00\x00\x00\x00\x93\xe0\x15\xf5E\x9e\xd2?hs\xd6\xcdA;\xce?\xd08\xae\xf1\xe3\xd4\xb9?\x00\x00\x00\x00\x00\x00\x00\x00\xaf\xe6BC\xe0s\xd3?HJN\xdb_\x97\xb1?\xad;B\xb3H$\xc6?$\xdc@\xf4[\x02\xc1?\x8e^`V\xb3/\xcc?\xb0ZE\xcc6\x82\xc6?\xcb\x8af\xce\x88\x9b\xdd?\xc2q\x91\x8c@X\xd5? \xc9\x01\xadE\xe0\xa0?\x9egBc"\x95\xd4?\x1d~\x9c\xcfk\xca\xc7?\x00\x7f\xd6\x81\xf4f\xdf?=$}\x90\x858\xda?\x00\x00\x00\x00\x00\x00\x00\x00\xc8\x97\x89y>~\xd2?)\\?\xdau\x05\xd0?\x88\xaf[\xd0\x82E\xc0?\x00\x00\x00\x00\x00\x00\x00\x00G6+\x97a\x17\xd3?\xb8\x06B\tf\x17\xb2?N\xd1\x13l\xb6`\xc5?r!\x01\x00\x00tr"\x01\x00\x00ba]r#\x01\x00\x00h\nh\x0b}r$\x01\x00\x00(h\rh\x0eh\x0fK\x00\x85r%\x01\x00\x00h\x11\x87r&\x01\x00\x00Rr\'\x01\x00\x00(K\x01K\x05\x85r(\x01\x00\x00h\x18\x89]r)\x01\x00\x00(h\x1cj\x0c\x01\x00\x00j\r\x01\x00\x00j\x0e\x01\x00\x00j\x0f\x01\x00\x00etr*\x01\x00\x00bh"Nu\x86r+\x01\x00\x00Rr,\x01\x00\x00a}r-\x01\x00\x00hW}r.\x01\x00\x00(hYj\x05\x01\x00\x00hZ]r/\x01\x00\x00}r0\x01\x00\x00(h]j\x1f\x01\x00\x00h^h_K\x00K\x05K\x01\x87r1\x01\x00\x00Rr2\x01\x00\x00uaustr3\x01\x00\x00bhchdhehfhg}r4\x01\x00\x00ubh\x03)\x81r5\x01\x00\x00}r6\x01\x00\x00(h\x06h\x07)\x81r7\x01\x00\x00(]r8\x01\x00\x00(h\nh\x0b}r9\x01\x00\x00(h\rh\x0eh\x0fK\x00\x85r:\x01\x00\x00h\x11\x87r;\x01\x00\x00Rr<\x01\x00\x00(K\x01K\x05\x85r=\x01\x00\x00h\x18\x89]r>\x01\x00\x00(h\x1cX\r\x00\x00\x00mean_recalledr?\x01\x00\x00X\x11\x00\x00\x00mean_not_recalledr@\x01\x00\x00X\x0c\x00\x00\x00std_recalledrA\x01\x00\x00X\x10\x00\x00\x00std_not_recalledrB\x01\x00\x00etrC\x01\x00\x00bh"Nu\x86rD\x01\x00\x00RrE\x01\x00\x00h\nh\x0b}rF\x01\x00\x00(h\rh\x0eh\x0fK\x00\x85rG\x01\x00\x00h\x11\x87rH\x01\x00\x00RrI\x01\x00\x00(K\x01K\x12\x85rJ\x01\x00\x00h\x18\x89]rK\x01\x00\x00(h+h,h-h.G\x7f\xf8\x00\x00\x00\x00\x00\x00h/h0h1h2h3h4h5h6h7h8h9h:h;etrL\x01\x00\x00bh"h=u\x86rM\x01\x00\x00RrN\x01\x00\x00e]rO\x01\x00\x00h\x0eh\x0fK\x00\x85rP\x01\x00\x00h\x11\x87rQ\x01\x00\x00RrR\x01\x00\x00(K\x01K\x05K\x12\x86rS\x01\x00\x00hG\x89B\xd0\x02\x00\x00\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x007@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00.@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00.@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00F@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00*@\xf2\x9c,Fv\x7f\x04\xc0\xaa\x8f\xce\xf1\xa8,\x08\xc0\x00X\xddh)\xa4\x04\xc0\xc2\xf6\x83\x8e\xb9\xc1\x03\xc0h)\xb3y\x8c\x9d\x03\xc0<\xb9\xd4\xb6\x9e\x94\x01\xc0D\xa36\xdeZO\x04\xc0\xfe\xe7lo|h\x05\xc0ea\xa7q\xb3\xca\x04\xc0L\xda\x9f\xf8\xdfg\x05\xc0W\xcfr\x08L\xd0\x02\xc06\xc2\xc1\x1f\n\xec\x03\xc0\x87W$\xf7\x99\xbe\x03\xc0\xba\x06\xa8U\x9f\xee\x02\xc0\x18]d#X\xce\xfc\xbfOe\xb8\xd7\xb5\xe6\x01\xc0g\xe0\x94\x1d\x89\x88\xff\xbf\x15\xc1D,\x83\xf8\x03\xc0O\x02\xf5bk\x90\x04\xc0\x8d\x9cRF\x870\x08\xc0pk\xcf52\x91\x04\xc0DC\xde\xe2\xd9\xc6\x03\xc0\xe8\x1e\x91^\xeek\x03\xc0\xee\x17\x88\x8d\x1d\xaf\x01\xc05\xbcO\xa3\x12_\x04\xc0\xa0\x84T\xccq\x9e\x05\xc0G\x95y0\xe7\xcd\x04\xc0[\t\x93\x95\x91\x90\x05\xc0|E\x04\xfd\x80\xba\x02\xc0\x07`\xcc\xfd\x8b\x0b\x04\xc0\xdd\xde\x83\xb3\xff\xc7\x03\xc0\xd7\xddI\x82;?\x03\xc0\xa7\x98qie\x9e\xfc\xbf\xa7\xfa\x08d\xc7\xec\x01\xc0\x1a\x87\x1a\xe7h\x96\xff\xbf\xe4:l\xde\'\xfe\x03\xc0\x99\xaf\xb1|\xf2j\xc4?\xdd\xbc\xads\x9e\xf8\xc8?\r\x0e\\\xd0r\xae\xcb?S\x04\x06\nQ\xee\xd7?"\xf8\x1b\xa3\xfc\xa6\xd1?\x00\xa0\x07I`\xfd\xa8?U\xa2\x89\x8e\xc9\x94\xd2? \xe4\xf8~\xb1\xda\xc9?\x9a&\x8e$\xf8\xd7\xde?u\x14\x93\x9c\\\x8a\xde?\x00\x00\x00\x00\x00\x00\x00\x00\xc7 \x86\x95\xe0\x9b\xd0?\xe5\xb0\xc6?{\xb4\xcd?(\x1dF\xd2\x12R\xc1?\x00\x00\x00\x00\x00\x00\x00\x00g1\xfc\x07R\xec\xd8?\xd0%\x9d:N$\xb2?\xa5.\xfe\xd8\xda\xe3\xcc?b\x99\x19\xef\x88M\xc5?<(\xc8\xbd\x1dc\xc7?mK\x8eO#L\xcb?\xac\xf3H;\x1ct\xd7?\x88\x01\xdf\x82X\x9c\xcf?\x80\xf7\xe9\xa7H\xcd\xab?9\xbf<6\xb0o\xd3?\xf2<\xcb2|\xec\xc9?\xe1`\x8d\xeb\xfec\xdf?f\x9b\x88\xf5\x04>\xde?\x00\x00\x00\x00\x00\x00\x00\x00\x10#\xad\xf4 V\xd0?F\x92\\uw\xcc\xce?\x90w\x0e\x12\x1f\xe5\xc3?\x00\x00\x00\x00\x00\x00\x00\x00\xc8\xb0\x08\xe7p_\xd8?\x90\xe3\x7f\xaa15\xa9?\xae\x0f\xdfo\x93\x1d\xce?rT\x01\x00\x00trU\x01\x00\x00ba]rV\x01\x00\x00h\nh\x0b}rW\x01\x00\x00(h\rh\x0eh\x0fK\x00\x85rX\x01\x00\x00h\x11\x87rY\x01\x00\x00RrZ\x01\x00\x00(K\x01K\x05\x85r[\x01\x00\x00h\x18\x89]r\\\x01\x00\x00(h\x1cj?\x01\x00\x00j@\x01\x00\x00jA\x01\x00\x00jB\x01\x00\x00etr]\x01\x00\x00bh"Nu\x86r^\x01\x00\x00Rr_\x01\x00\x00a}r`\x01\x00\x00hW}ra\x01\x00\x00(hYj8\x01\x00\x00hZ]rb\x01\x00\x00}rc\x01\x00\x00(h]jR\x01\x00\x00h^h_K\x00K\x05K\x01\x87rd\x01\x00\x00Rre\x01\x00\x00uaustrf\x01\x00\x00bhchdhehfhg}rg\x01\x00\x00ubes.'

---

**R1385E** - 5


In [ ]:
byte_alpha = b'\x80\x03}q\x00X\x06\x00\x00\x00R1385Eq\x01]q\x02(cpandas.core.frame\nDataFrame\nq\x03)\x81q\x04}q\x05(X\x05\x00\x00\x00_dataq\x06cpandas.core.internals.managers\nBlockManager\nq\x07)\x81q\x08(]q\t(cpandas.core.indexes.base\n_new_Index\nq\ncpandas.core.indexes.base\nIndex\nq\x0b}q\x0c(X\x04\x00\x00\x00dataq\rcnumpy.core.multiarray\n_reconstruct\nq\x0ecnumpy\nndarray\nq\x0fK\x00\x85q\x10C\x01bq\x11\x87q\x12Rq\x13(K\x01K\x05\x85q\x14cnumpy\ndtype\nq\x15X\x02\x00\x00\x00O8q\x16K\x00K\x01\x87q\x17Rq\x18(K\x03X\x01\x00\x00\x00|q\x19NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?tq\x1ab\x89]q\x1b(X\x05\x00\x00\x00countq\x1cX\r\x00\x00\x00mean_recalledq\x1dX\x11\x00\x00\x00mean_not_recalledq\x1eX\x0c\x00\x00\x00std_recalledq\x1fX\x10\x00\x00\x00std_not_recalledq etq!bX\x04\x00\x00\x00nameq"Nu\x86q#Rq$h\nh\x0b}q%(h\rh\x0eh\x0fK\x00\x85q&h\x11\x87q\'Rq((K\x01K\x0f\x85q)h\x18\x89]q*(X\x06\x00\x00\x00insulaq+X\x10\x00\x00\x00superiortemporalq,X\x0e\x00\x00\x00middletemporalq-G\x7f\xf8\x00\x00\x00\x00\x00\x00X\x08\x00\x00\x00banksstsq.X\x18\x00\x00\x00rostralanteriorcingulateq/X\x0f\x00\x00\x00superiorfrontalq0X\x0c\x00\x00\x00temporalpoleq1X\x10\x00\x00\x00inferiortemporalq2X\x14\x00\x00\x00lateralorbitofrontalq3X\x10\x00\x00\x00parstriangularisq4X\x14\x00\x00\x00rostralmiddlefrontalq5X\n\x00\x00\x00entorhinalq6X\x0f\x00\x00\x00parsopercularisq7X\x08\x00\x00\x00fusiformq8etq9bh"X\x04\x00\x00\x00areaq:u\x86q;Rq<e]q=h\x0eh\x0fK\x00\x85q>h\x11\x87q?Rq@(K\x01K\x05K\x0f\x86qAh\x15X\x02\x00\x00\x00f8qBK\x00K\x01\x87qCRqD(K\x03X\x01\x00\x00\x00<qENNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tqFb\x89BX\x02\x00\x00\x00\x00\x00\x00\x00\x00@@\x00\x00\x00\x00\x00\x003@\x00\x00\x00\x00\x00\x00A@\x00\x00\x00\x00\x00\x003@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00(@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x001@\x00\x00\x00\x00\x00\x00$@\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf0?\x04\xf3\x1dj[\xc0\x05\xc0\x8d\x85\xf7R \xa6\t\xc0<|\xf5i\xf3\xc0\t\xc0\x00\x84\xce\xeb\x8e\xbc\x07\xc0h\xc5\x18v\xd9\x8d\x08\xc0\xa2&\xead\xba\xc9\x05\xc0\x97\xca\x0c\xf4\xcb\xc5\x06\xc0\xfaw\xa5\x14\xd3s\n\xc0\xed\xe8\xec\x85\x07\x19\t\xc0Yq\xdf\xe4\xa9\x1c\x06\xc0\xf2\x94\xd7\xa4\xd6\x9b\x07\xc0\xed\x80J\xe4\x84u\x06\xc0\x08\xf8$x\xb1\xa7\x06\xc0\xc0\xd8\xa6\x82\xacv\x06\xc00\tNY@\x82\x08\xc0\xd1H\xc9O\xa6\xc5\x05\xc0[\xabo\x8b\xd6\xbc\t\xc0\x0e\xca\xee\xdc\xb9\xa4\t\xc0Ht\xff\xa6\x82\x8e\x07\xc0\x00T@\xe1\xbe\x8b\x08\xc0\xc8\x07D\x9f\xfd\x8f\x05\xc0\x83\xc3\xda\x13\'\xa8\x06\xc0\xef_\xbe\x8fH\x83\n\xc0\xc5\x83g\x949-\t\xc0/P\xb3\xb1\xde\x13\x06\xc0I\x05K\x8cf\x92\x07\xc0>\xb9\xc0\x87\xe1\x1d\x06\xc0\x8d|\x06\x8f?\xc8\x06\xc0\xc8\xae\xf5.\xf0\xc9\x06\xc0\x03>\xd3\xcf6\x99\x08\xc0\xea\xc8!\xec\xc37\xd1?~[:q\x92\xf6\xcc?\xb5\xb9\x16C\x0f\xce\xcd??\x8d\xb6x+\xf7\xce?k\xa2\x9d{9c\xc5?k\xd7n\x90\x0e\xdf\xa9?\x8d\xc2\xe6\xf7I\xe9\xc5?\xb8A_\xeeM\'\xaf?\x96\xf2\xca\xf2iy\xc6?\x04\xe8\x80\xf1\xee\xe3\xc1?\xca(%\xe3\x9eJ\xc5?\x00\x00\x00\x00\x00\x00\x00\x00\xe0\x8a9\xb5Pd\xc5?`\x14\xd0\x97\xef\xa3\xbb?\x00\x00\x00\x00\x00\x00\x00\x00\xed\x83\xdfuG\xcf\xd0?;: SD\x85\xcd?\xddC\xf9\xe3\xd1\x84\xcc?\x08m\xe0I\x83\x88\xcf?P\x14\x91s\xc7R\xc5?:\xbb_fML\xa6?\xaf\x11k\x16\n%\xc6?;\x89n\x03\xfcE\xb2?f\xb8N\xbe\x8d\x9e\xc5?d\xcd\xb4\xd6\x94\x83\xc0?\xbe\xdaE\x94\xcb\x0e\xc4?\x00\x00\x00\x00\x00\x00\x00\x00\xcah\x02\'2\x80\xc4?\xd8\xab+>\xf0\xc5\xc0?\x00\x00\x00\x00\x00\x00\x00\x00qGtqHba]qIh\nh\x0b}qJ(h\rh\x0eh\x0fK\x00\x85qKh\x11\x87qLRqM(K\x01K\x05\x85qNh\x18\x89]qO(h\x1ch\x1dh\x1eh\x1fh etqPbh"Nu\x86qQRqRa}qSX\x06\x00\x00\x000.14.1qT}qU(X\x04\x00\x00\x00axesqVh\tX\x06\x00\x00\x00blocksqW]qX}qY(X\x06\x00\x00\x00valuesqZh@X\x08\x00\x00\x00mgr_locsq[cbuiltins\nslice\nq\\K\x00K\x05K\x01\x87q]Rq^uaustq_bX\x04\x00\x00\x00_typq`X\t\x00\x00\x00dataframeqaX\t\x00\x00\x00_metadataqb]qcX\x05\x00\x00\x00attrsqd}qeubh\x03)\x81qf}qg(h\x06h\x07)\x81qh(]qi(h\nh\x0b}qj(h\rh\x0eh\x0fK\x00\x85qkh\x11\x87qlRqm(K\x01K\x05\x85qnh\x18\x89]qo(h\x1cX\r\x00\x00\x00mean_recalledqpX\x11\x00\x00\x00mean_not_recalledqqX\x0c\x00\x00\x00std_recalledqrX\x10\x00\x00\x00std_not_recalledqsetqtbh"Nu\x86quRqvh\nh\x0b}qw(h\rh\x0eh\x0fK\x00\x85qxh\x11\x87qyRqz(K\x01K\x0f\x85q{h\x18\x89]q|(h+h,h-G\x7f\xf8\x00\x00\x00\x00\x00\x00h.h/h0h1h2h3h4h5h6h7h8etq}bh"h:u\x86q~Rq\x7fe]q\x80h\x0eh\x0fK\x00\x85q\x81h\x11\x87q\x82Rq\x83(K\x01K\x05K\x0f\x86q\x84hD\x89BX\x02\x00\x00\x00\x00\x00\x00\x00\x00@@\x00\x00\x00\x00\x00\x003@\x00\x00\x00\x00\x00\x00A@\x00\x00\x00\x00\x00\x003@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00(@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x001@\x00\x00\x00\x00\x00\x00$@\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf0?\x11\x02w\x9e.%\x05\xc0\xa7\x19\x8a\r\x80\xe4\x08\xc0\xe5\x05=\xd38\xec\x08\xc0$o\xba\xef\x1b\xd6\x06\xc0\xb2\xa4\x9dM\xaeD\x07\xc0\xc6\xcf+\xaat\xdd\x04\xc0]i+\xe4\xe9\x9a\x05\xc0D\xacz\xdb\xa7\x0f\n\xc0p\x8a\x1e\xb0\xae\xa0\x08\xc03\x84W\xd5CI\x05\xc0\x8b\xafZ\xcfw\xaf\x06\xc0\xa8\xcd\xa1\x96J\xa7\x05\xc0\xbfA\x9a\xa4\xa1\xe5\x05\xc0\x92O\x1a\xce\x96\xe4\x05\xc0~\xe3\xb3\xe1\xc3\xac\x07\xc02s\xe8l};\x05\xc0%{\xd5;\xf5\xce\x08\xc0\xf1\xe0\x99\x01\x1e\xf8\x08\xc0\xc2.\xa3\xd2\xcf\xeb\x06\xc0Zs\x16J\xd7c\x07\xc0\xcc\'\xb7bj\xe7\x04\xc0$\xf8\xeb\x95\xa1\xb6\x05\xc0\xaf\x95\te\x07 \n\xc0\xc9\xc6\x8c\xc0,\xb3\x08\xc0\xc7G\x1c\xb9\x02_\x05\xc0\xed\x95\xd3L\xe3\xc9\x06\xc0F\x11R\xe8\xa0\xcd\x05\xc0\xe8\xd5\xf3~Q\x12\x06\xc0n\x98\r\xf1\x16\xe5\x05\xc0d\xe8\xf9\xd6\x0b\xc9\x07\xc0\x05\x1b\xdc.\x9b\xb7\xd0?\x1f\xdf\xc3RM"\xd0?\xb1xv<I\xa5\xcf?)\xd5\xf7\x9bb\x12\xd1?\x90\x9e\xd9\xb5*\x93\xc5?\xebF\x16X\x8d\x12\xa8?\xeb\xfc)c\xb9\n\xc6?BPZg\x9e+\xb4?qPF1\x17\x0b\xc5?\xd6\x90\xe0f#?\xc2?\r\x9fM\xe8S\xae\xc1?\x00\x00\x00\x00\x00\x00\x00\x00\xc5\xaa\xaey\xd5U\xc5?\xe0\xde\x94 E\x9e\xbf?\x00\x00\x00\x00\x00\x00\x00\x00OI\xb3\x94\x9fd\xd0?\xde\x9a#\x0f\xd4x\xd0?r?\x01J\x15\xbb\xce?~O=\x87\xa2\xc2\xd0?g7lF\x00\xe5\xc5?)\x85\xfe*\x18\xc8\xa8?\x97b\xa1\x9cn\xe9\xc6?\xd4\xd9\x95\xe5>T\xb3?V\xe88\xe4*d\xc3?d\x91\n\x99+\x0f\xc2?1\xfc\xcb\xb8\xf3`\xc2?\x00\x00\x00\x00\x00\x00\x00\x00a,\xd1\xfa\xc6\xbb\xc0?\xd0.\x17\xf1\x97\x7f\xbd?\x00\x00\x00\x00\x00\x00\x00\x00q\x85tq\x86ba]q\x87h\nh\x0b}q\x88(h\rh\x0eh\x0fK\x00\x85q\x89h\x11\x87q\x8aRq\x8b(K\x01K\x05\x85q\x8ch\x18\x89]q\x8d(h\x1chphqhrhsetq\x8ebh"Nu\x86q\x8fRq\x90a}q\x91hT}q\x92(hVhihW]q\x93}q\x94(hZh\x83h[h\\K\x00K\x05K\x01\x87q\x95Rq\x96uaustq\x97bh`hahbhchd}q\x98ubh\x03)\x81q\x99}q\x9a(h\x06h\x07)\x81q\x9b(]q\x9c(h\nh\x0b}q\x9d(h\rh\x0eh\x0fK\x00\x85q\x9eh\x11\x87q\x9fRq\xa0(K\x01K\x05\x85q\xa1h\x18\x89]q\xa2(h\x1cX\r\x00\x00\x00mean_recalledq\xa3X\x11\x00\x00\x00mean_not_recalledq\xa4X\x0c\x00\x00\x00std_recalledq\xa5X\x10\x00\x00\x00std_not_recalledq\xa6etq\xa7bh"Nu\x86q\xa8Rq\xa9h\nh\x0b}q\xaa(h\rh\x0eh\x0fK\x00\x85q\xabh\x11\x87q\xacRq\xad(K\x01K\x0f\x85q\xaeh\x18\x89]q\xaf(h+h,h-G\x7f\xf8\x00\x00\x00\x00\x00\x00h.h/h0h1h2h3h4h5h6h7h8etq\xb0bh"h:u\x86q\xb1Rq\xb2e]q\xb3h\x0eh\x0fK\x00\x85q\xb4h\x11\x87q\xb5Rq\xb6(K\x01K\x05K\x0f\x86q\xb7hD\x89BX\x02\x00\x00\x00\x00\x00\x00\x00\x00@@\x00\x00\x00\x00\x00\x003@\x00\x00\x00\x00\x00\x00A@\x00\x00\x00\x00\x00\x003@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00(@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x001@\x00\x00\x00\x00\x00\x00$@\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf0?r\xda\x86\xdc\xe9\x89\x06\xc0\xde\xf3\xd4S\x90#\t\xc0\xe2\xc5\xb9X\xec\xf4\t\xc0=\x12\x1f0\xc8%\t\xc0\x87sF\xd6\x01n\t\xc0\xf4j\x7f\n\xa2b\x07\xc0m\xbdx\x0f\x88\xfe\x07\xc0k\x92D\x8e\xea\xed\t\xc0\xd5\xb4\xc3~\xbd^\t\xc0U\x13j\xbdp\xf1\x06\xc0E\x18\xe9[\r3\x07\xc0l\xfe\x0fO\x0b\xb6\x06\xc0\xdc\xa4\xa3\x91\x97\xbb\x07\xc0U\xf5\x06\xb0\x14\x93\x06\xc0&70\xba\xcc\xf1\t\xc0\xb2\x0b&\x8a\x86\x8c\x06\xc0-\xa8\x86L:\t\t\xc0~\xc0;\x87\xd6\xf4\t\xc0\xde\x9d\x18\xa4s\'\t\xc0\xbeH\xa3\xc6\x93v\t\xc0\xa9\x1e\xa3!\xcdF\x07\xc00\x059g8\x07\x08\xc0,!Q\x89~\xe1\t\xc0\xc6\xa0\r\xf7\xe5R\t\xc0\xe6u\x1em\xd0\xff\x06\xc0\xa8:\x1d<\xe6H\x07\xc0_\x91\x0e\xc8g\x88\x06\xc0\xa4\x0fB\xb1\xcf\x9c\x07\xc0\x14\xc1\xd3\x9b\x82P\x06\xc0\x9d0\xd7\xae\x8a\xcf\t\xc0\x88\x9bZV\x1b\xa4\xd2?\xf2o\xa9\xa4\x9eC\xcf?Z7\xd2a\x94z\xd1?\x8f4\xdc\xcd{2\xd1?\xbc;\x984\x1dP\xcd?%\x0bG\xf4\xde\xd0\xc2?\x00*o\x9a\x1e\x07\xd2?\xad\x8c5\xf0\xd4\x1f\xbd?\x988\x82X\x1bt\xcb?"\xc6\xd1Jl\xb7\xc5?r\xfe\xfcT:\xb3\xc7?\x00\x00\x00\x00\x00\x00\x00\x00\xff\x8d^}Zo\xc1?\x00K\x9aCe\xc3\xc5?\x00\x00\x00\x00\x00\x00\x00\x00\xb4U\x19\xc0K\xfa\xd2?AE\x8d\xdf\xb1\xa0\xcf?|\xa5\xa8\x92\'e\xd1?\x1f\x134k\xd1C\xd1?|n\x13\xf3j\x9f\xd0?\x85\xa3_)V"\xc3?\x17\xa6\xb2\xc1\xc3\xcc\xd1?\x10\x02`\x17\x10\xbd\xbb?rO\xc6\x91!)\xcc?\x95T\x84\x11\xf04\xc5?K\xe1_\xe9\xe0\x16\xc9?\x00\x00\x00\x00\x00\x00\x00\x00\x88\x10aw\x99c\xbf?(\xfe,t\x19\xd0\xc5?\x00\x00\x00\x00\x00\x00\x00\x00q\xb8tq\xb9ba]q\xbah\nh\x0b}q\xbb(h\rh\x0eh\x0fK\x00\x85q\xbch\x11\x87q\xbdRq\xbe(K\x01K\x05\x85q\xbfh\x18\x89]q\xc0(h\x1ch\xa3h\xa4h\xa5h\xa6etq\xc1bh"Nu\x86q\xc2Rq\xc3a}q\xc4hT}q\xc5(hVh\x9chW]q\xc6}q\xc7(hZh\xb6h[h\\K\x00K\x05K\x01\x87q\xc8Rq\xc9uaustq\xcabh`hahbhchd}q\xcbubes.'

---

**R1189M** - 6


In [ ]:
byte_alpha = b'\x80\x03}q\x00X\x06\x00\x00\x00R1189Mq\x01]q\x02(cpandas.core.frame\nDataFrame\nq\x03)\x81q\x04}q\x05(X\x05\x00\x00\x00_dataq\x06cpandas.core.internals.managers\nBlockManager\nq\x07)\x81q\x08(]q\t(cpandas.core.indexes.base\n_new_Index\nq\ncpandas.core.indexes.base\nIndex\nq\x0b}q\x0c(X\x04\x00\x00\x00dataq\rcnumpy.core.multiarray\n_reconstruct\nq\x0ecnumpy\nndarray\nq\x0fK\x00\x85q\x10C\x01bq\x11\x87q\x12Rq\x13(K\x01K\x05\x85q\x14cnumpy\ndtype\nq\x15X\x02\x00\x00\x00O8q\x16K\x00K\x01\x87q\x17Rq\x18(K\x03X\x01\x00\x00\x00|q\x19NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?tq\x1ab\x89]q\x1b(X\x05\x00\x00\x00countq\x1cX\r\x00\x00\x00mean_recalledq\x1dX\x11\x00\x00\x00mean_not_recalledq\x1eX\x0c\x00\x00\x00std_recalledq\x1fX\x10\x00\x00\x00std_not_recalledq etq!bX\x04\x00\x00\x00nameq"Nu\x86q#Rq$h\nh\x0b}q%(h\rh\x0eh\x0fK\x00\x85q&h\x11\x87q\'Rq((K\x01K\x0c\x85q)h\x18\x89]q*(X\x17\x00\x00\x00caudalanteriorcingulateq+X\x0f\x00\x00\x00superiorfrontalq,X\x14\x00\x00\x00rostralmiddlefrontalq-X\x13\x00\x00\x00caudalmiddlefrontalq.X\x0b\x00\x00\x00paracentralq/X\n\x00\x00\x00precentralq0X\x0b\x00\x00\x00postcentralq1X\x10\x00\x00\x00parstriangularisq2X\x12\x00\x00\x00posteriorcingulateq3X\x10\x00\x00\x00superiorparietalq4X\r\x00\x00\x00supramarginalq5X\x06\x00\x00\x00insulaq6etq7bh"X\x04\x00\x00\x00areaq8u\x86q9Rq:e]q;h\x0eh\x0fK\x00\x85q<h\x11\x87q=Rq>(K\x01K\x05K\x0c\x86q?h\x15X\x02\x00\x00\x00f8q@K\x00K\x01\x87qARqB(K\x03X\x01\x00\x00\x00<qCNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tqDb\x89B\xe0\x01\x00\x00\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00&@\x00\x00\x00\x00\x00\x00,@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x002@\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x14@w=\xac\xb3~\x18\xfe\xbf\x8co8\x1c\xfe\xa1\xfc\xbf\xfb,\xae\x00{5\xfe\xbf\x00h\x81\xc3MO\xfb\xbf\xe4{m\xddMb\xfe\xbf\x15\x8b\x00\xf9r\xc6\xff\xbf\xaa[\xe0\xc0"\x99\x00\xc0:\xbd>nk\x17\x02\xc0\xc3\x8cj\xd3\xfe2\xf9\xbfcn=\xdd\x86T\x04\xc0B:\xdb \xdf\x08\x08\xc0\x05\xd3\xceyU\x01\xfa\xbf\xbb^\xba\x04\x17\x94\xfd\xbf\xc1\x13!\xc2p\x12\xfd\xbf\x97}u\x02\x8d\x96\xfe\xbf\xbc\xd8\xa1hB\x84\xfb\xbf\x8d\xff\x9d\x87\x06g\xfe\xbf"\x82(\xbf\x92\xcb\xff\xbfL\xd8\xc6\x1b\x8f\x96\x00\xc0\xdb7@\xae\x96\x10\x02\xc0\xae\xe6\xa0\xa2?H\xf9\xbf?\x8c\xf4>\xadT\x04\xc0\x02\x07"\xae\x87\x99\x07\xc0\xa04\xb9b\xc4\xff\xf9\xbf\xea\xbb\xfc\x1b\xab\x06\xe2?ock0\xf3\xd9\xd1?o\x912\xd9\x08\xa1\xe2?H\xcd\x16\xc5*\x8a\xe6?\x0cA\n\xcc\xffQ\xce?\x8d.\x9fu\xd5\xbc\xdc?\x7f!mnH\'\xe6?$5\xa4(k6\xbc?e\x85\x04fC\xa5\xe0?O\x8a\x14\xe0<\xb5\xd6?F3S\x04@\xd0\xd4?\xe2 7\x821\xff\xd8?\x86\xcb\xcd\x85\x88\x91\xe1?E?\x9e\xe0M\xae\xd0?hnX\x10!\xdf\xe2?!\xcd{r+n\xe6?K\xdf\xde\x86b\x1c\xce?|\xb9\xfc\x1co\x0e\xdc?d5VkM\x98\xe5?o\x8f\xe65\x11\x01\xb7?1\x8fo\xea\xa3\x83\xe0?l+7d\xce\xc5\xd6?\xe6\x9c\xc6\xa3\xb3S\xd4?\xf6\xc6\xbbRa=\xda?qEtqFba]qGh\nh\x0b}qH(h\rh\x0eh\x0fK\x00\x85qIh\x11\x87qJRqK(K\x01K\x05\x85qLh\x18\x89]qM(h\x1ch\x1dh\x1eh\x1fh etqNbh"Nu\x86qORqPa}qQX\x06\x00\x00\x000.14.1qR}qS(X\x04\x00\x00\x00axesqTh\tX\x06\x00\x00\x00blocksqU]qV}qW(X\x06\x00\x00\x00valuesqXh>X\x08\x00\x00\x00mgr_locsqYcbuiltins\nslice\nqZK\x00K\x05K\x01\x87q[Rq\\uaustq]bX\x04\x00\x00\x00_typq^X\t\x00\x00\x00dataframeq_X\t\x00\x00\x00_metadataq`]qaX\x05\x00\x00\x00attrsqb}qcubh\x03)\x81qd}qe(h\x06h\x07)\x81qf(]qg(h\nh\x0b}qh(h\rh\x0eh\x0fK\x00\x85qih\x11\x87qjRqk(K\x01K\x05\x85qlh\x18\x89]qm(h\x1cX\r\x00\x00\x00mean_recalledqnX\x11\x00\x00\x00mean_not_recalledqoX\x0c\x00\x00\x00std_recalledqpX\x10\x00\x00\x00std_not_recalledqqetqrbh"Nu\x86qsRqth\nh\x0b}qu(h\rh\x0eh\x0fK\x00\x85qvh\x11\x87qwRqx(K\x01K\x0c\x85qyh\x18\x89]qz(h+h,h-h.h/h0h1h2h3h4h5h6etq{bh"h8u\x86q|Rq}e]q~h\x0eh\x0fK\x00\x85q\x7fh\x11\x87q\x80Rq\x81(K\x01K\x05K\x0c\x86q\x82hB\x89B\xe0\x01\x00\x00\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00&@\x00\x00\x00\x00\x00\x00,@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x002@\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x14@\x0f\xf0\x12y`0\x00\xc0\xd1\x9e\x9e^zZ\xfe\xbfQ\x88\xa1\xd2\x9d\xf2\xfc\xbf~\xeet\xd2\x0e\xa7\xfb\xbf\xf4\x07hMH\xb5\xfe\xbf\xb2\x103\x8e\x88k\xfe\xbf\x0e.I\xechy\xfe\xbfk\x85\xc3\x16\xfc\xc7\x00\xc00\x93p\x06\xffQ\xfa\xbf4M\x18\x98X\xd9\x02\xc0\x1b\x7f9D\xa1\xcc\x06\xc0\xd0Ri>\xc0i\xf9\xbf\xb9\xa3\xa5\x0e\xdc\x02\x00\xc0\x81\x87x9\x1b\xaa\xfd\xbfE\x8ea\n\xfc\xd9\xfc\xbf\xf24\x82\x9c!>\xfb\xbf\xc8>\xe1@\xe3\x9e\xfe\xbf\x8cP\'\x08\xdc?\xfe\xbf\x9e(\x11X\x97N\xfe\xbf\xaf\x9d\xd6\xb1\xa6\xa2\x00\xc0\xb5\xcetY\x82@\xfa\xbf\x03\x98|V\x8bR\x02\xc0n\xc9\x00{\xac\x8b\x06\xc0\x0f\x98&M\xb1\x03\xf9\xbf\xdbGU\x12\xb5T\xe2?\xf5\x03\xdfMC\xa5\xd6?\xca8\x0f\';D\xe1?\x90\x13@\x81\x8c\x80\xe3?|\x10t\xb9\x19\x88\xd2?\xcdLV<m6\xd7?$:83\xab\x91\xe4?)R\xf8W\xf6\x0e\xba?\xe3\xf2\xb1\'\x80\x9d\xdc?\x02\xa7\x94-\x1a\x97\xd3?.[O\x96!\x1a\xd4?4\x83\xf7\x1aKW\xdc?\xf6\xe5\xa0x%\xeb\xe0?\x98\x1f\x1e{,\xce\xd6?\xb4LW\x8d:\x11\xe1?\xfb\xef\x01\r\xe8\x17\xe3?\xac\x15\xe4#:\xf0\xd2?3hQ)\xdbe\xd6?\xa4\xb9\r\xc7A\xaf\xe4?\xce\xe1>}\\P\xb4?V\x0cY\x10\x97*\xdd?\x1a\xc5\x9f\xac\x92\xea\xd2?\xcfr\xa9\xc5\x9dd\xd3?n\x91\x9a\x84\xf3\xac\xdc?q\x83tq\x84ba]q\x85h\nh\x0b}q\x86(h\rh\x0eh\x0fK\x00\x85q\x87h\x11\x87q\x88Rq\x89(K\x01K\x05\x85q\x8ah\x18\x89]q\x8b(h\x1chnhohphqetq\x8cbh"Nu\x86q\x8dRq\x8ea}q\x8fhR}q\x90(hThghU]q\x91}q\x92(hXh\x81hYhZK\x00K\x05K\x01\x87q\x93Rq\x94uaustq\x95bh^h_h`hahb}q\x96ubh\x03)\x81q\x97}q\x98(h\x06h\x07)\x81q\x99(]q\x9a(h\nh\x0b}q\x9b(h\rh\x0eh\x0fK\x00\x85q\x9ch\x11\x87q\x9dRq\x9e(K\x01K\x05\x85q\x9fh\x18\x89]q\xa0(h\x1cX\r\x00\x00\x00mean_recalledq\xa1X\x11\x00\x00\x00mean_not_recalledq\xa2X\x0c\x00\x00\x00std_recalledq\xa3X\x10\x00\x00\x00std_not_recalledq\xa4etq\xa5bh"Nu\x86q\xa6Rq\xa7h\nh\x0b}q\xa8(h\rh\x0eh\x0fK\x00\x85q\xa9h\x11\x87q\xaaRq\xab(K\x01K\x0c\x85q\xach\x18\x89]q\xad(h+h,h-h.h/h0h1h2h3h4h5h6etq\xaebh"h8u\x86q\xafRq\xb0e]q\xb1h\x0eh\x0fK\x00\x85q\xb2h\x11\x87q\xb3Rq\xb4(K\x01K\x05K\x0c\x86q\xb5hB\x89B\xe0\x01\x00\x00\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00&@\x00\x00\x00\x00\x00\x00,@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x002@\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x14@\xf7\xf2\xd7\xda5;\x00\xc0\xf3\x11\xeaxP/\xfe\xbfI\xc3\xd2(\x15)\xfc\xbf\x16\x1aH\r\x03\x1f\xfb\xbf\xd0\x99\xfa\x88^^\xfd\xbf\x1f\x88D\x19\xe8d\xfc\xbfy\xea<g\xf3W\xfb\xbfU:\x8c,MB\x00\xc0\xdb\xcaB\x1a\x19G\xfa\xbf:9?\x86\x1e[\x01\xc0\x89~y\x8f\x8d\xbd\x04\xc0M\x9e\x88\xc8\xca"\xfb\xbf\x05\x87B\x86\x90\x93\x00\xc0]\x94\xc0N\rl\xfd\xbfs\xaa?*\x94\xd9\xfb\xbf\x166Cc\xd3\xc1\xfa\xbf\xc7EJ\xc0J2\xfd\xbf,I\xbc\xb6\x1e\x9a\xfc\xbf\x97\xd2\xae\x03|.\xfb\xbf1\x13$\xe2\\\xaf\xff\xbf~\xa9\x05y_\xe2\xf9\xbf\xe8\xeb\xc0\xee\x07F\x01\xc0+\xb9X1o\xb9\x04\xc0\x82\x1f\xcf\xe2\x80\x8e\xfb\xbf\xfdw\xe0 \x0cM\xe2?s\x8f\xc3v\xfc\xdb\xd7?\xed#\xd0\x8c\x8f\x01\xdf?}\xe9\x9d\x92f\x1d\xe2?\xf0\x8dO \xd0`\xd6?\x97\xd3\x83+\x92@\xd4?\xe7V\xcf\xc7\xad^\xe1?\xcb\x96$\x88\x1a\xf9\xba?\x1e\xf4!\xeb\xc4\xf4\xd9?\xf8\xf9\xdf\xfe\xe9\x82\xd1?\xea$`\x004#\xd3?W\x15b\x9e\xafB\xd2?\x14d\x7f\x92\xbb2\xe3?*\xaa\x14M:n\xd7?g]\xb9\xf2\xf8m\xde?D\xb04(\x90,\xe1?\xc9w\xaf\x1c>\xee\xd5?8f\xc3\x804\x9d\xd4?s\xc40\x8f\xdd\xaf\xe1?\x8c\xf6^Q\x9dY\xbb?\xb3\xben\xb3\x80\xd4\xd8?\xb4\xfc\xb5u\x95\x1e\xd1?\x1b\xc9*b\xf28\xd2?\x17\xc7t\xfa\xd3\x06\xcf?q\xb6tq\xb7ba]q\xb8h\nh\x0b}q\xb9(h\rh\x0eh\x0fK\x00\x85q\xbah\x11\x87q\xbbRq\xbc(K\x01K\x05\x85q\xbdh\x18\x89]q\xbe(h\x1ch\xa1h\xa2h\xa3h\xa4etq\xbfbh"Nu\x86q\xc0Rq\xc1a}q\xc2hR}q\xc3(hTh\x9ahU]q\xc4}q\xc5(hXh\xb4hYhZK\x00K\x05K\x01\x87q\xc6Rq\xc7uaustq\xc8bh^h_h`hahb}q\xc9ubes.'

---

**R1108J** - 7


In [ ]:
# no relevant experiments for this subject

---

**R1390M** - 8


In [ ]:
byte_alpha = b'\x80\x03}q\x00X\x06\x00\x00\x00R1390Mq\x01]q\x02cpandas.core.frame\nDataFrame\nq\x03)\x81q\x04}q\x05(X\x05\x00\x00\x00_dataq\x06cpandas.core.internals.managers\nBlockManager\nq\x07)\x81q\x08(]q\t(cpandas.core.indexes.base\n_new_Index\nq\ncpandas.core.indexes.base\nIndex\nq\x0b}q\x0c(X\x04\x00\x00\x00dataq\rcnumpy.core.multiarray\n_reconstruct\nq\x0ecnumpy\nndarray\nq\x0fK\x00\x85q\x10C\x01bq\x11\x87q\x12Rq\x13(K\x01K\x05\x85q\x14cnumpy\ndtype\nq\x15X\x02\x00\x00\x00O8q\x16K\x00K\x01\x87q\x17Rq\x18(K\x03X\x01\x00\x00\x00|q\x19NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?tq\x1ab\x89]q\x1b(X\x05\x00\x00\x00countq\x1cX\r\x00\x00\x00mean_recalledq\x1dX\x11\x00\x00\x00mean_not_recalledq\x1eX\x0c\x00\x00\x00std_recalledq\x1fX\x10\x00\x00\x00std_not_recalledq etq!bX\x04\x00\x00\x00nameq"Nu\x86q#Rq$h\nh\x0b}q%(h\rh\x0eh\x0fK\x00\x85q&h\x11\x87q\'Rq((K\x01K\x0e\x85q)h\x18\x89]q*(X\x06\x00\x00\x00insulaq+X\x10\x00\x00\x00superiortemporalq,X\x10\x00\x00\x00inferiortemporalq-G\x7f\xf8\x00\x00\x00\x00\x00\x00X\x10\x00\x00\x00inferiorparietalq.X\x08\x00\x00\x00banksstsq/X\n\x00\x00\x00entorhinalq0X\x08\x00\x00\x00fusiformq1X\x0f\x00\x00\x00parahippocampalq2X\x07\x00\x00\x00unknownq3X\x0e\x00\x00\x00middletemporalq4X\x0b\x00\x00\x00postcentralq5X\r\x00\x00\x00supramarginalq6X\x10\x00\x00\x00lateraloccipitalq7etq8bh"X\x04\x00\x00\x00areaq9u\x86q:Rq;e]q<h\x0eh\x0fK\x00\x85q=h\x11\x87q>Rq?(K\x01K\x05K\x0e\x86q@h\x15X\x02\x00\x00\x00f8qAK\x00K\x01\x87qBRqC(K\x03X\x01\x00\x00\x00<qDNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tqEb\x89B0\x02\x00\x00\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00&@\x00\x00\x00\x00\x00\x00:@\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00(@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x00@\x87X%>\xa9\xd9\x07\xc0J\xee\xd3\xd4\x1e/\x05\xc0.\xf0f\xdd\x8fN\x06\xc0\x8c\xa1\xc4\x82\xf3\x01\x07\xc0\x0c\x0ed\xe3\xcd\xc4\x05\xc0\x14t\xe4\xc1,g\x04\xc0\xc1\xa8\x8b\xb8T\xc7\x04\xc0\x89O\xc8\x1b{\xac\x03\xc0\xe4\xb2\x0c\xd4OM\x04\xc0\xa8\xc3\xcafR\xcc\x06\xc0S\xb3o\xb0\x12\xd3\x05\xc0\x00\x96K\x17\xcd\xc7\x07\xc0\xb7\xca\xc6\xb3h#\x05\xc0h\x11,\xc6z\xc3\x04\xc00\xd9zyH\xcb\x07\xc0\xbe\x82\x95\xe6B+\x05\xc0J\x9ds\x0f\xaef\x06\xc0\xa6\xebU&.\x02\x07\xc0\xad\x9cW[\x86\xce\x05\xc0\xef{\xf5GA\xa9\x04\xc0\x02f4f)\x95\x04\xc0_\xa9y\xd8\x02\xa8\x03\xc0\x90\xe3\xc2(\xb3\x88\x04\xc0oU\xa3\xff\x1e\xed\x06\xc0s\xf1\x1f\xacT\x14\x06\xc0\xff6\x92\xeb\xa0\x95\x07\xc0\x9c\x04M+U*\x05\xc0\x04E\xab\xdc&\xe8\x04\xc0M\xa9\xd5RZK\xd1?1\x10\x13\xc4*\xe3\xd3?\xa0)\xa9\xc0\x89\x8c\xc4?\xfc \x82\x84\xd8\x9d\xcf?\xea\x9b\xbc\x88[\\\xc6?\xc0\x95\x93|sk\xb8?\x00\x00\x00\x00\x00\x00\x00\x00(\x16\x82\xbd\xceF\xbf?\xa0)"lY\x03\xa7?v\xba\xaeJ,\xe7\xc9?\xaf\xb2\xc3\xe6\x1c\xf1\xca?\xf0\xdc\xe5\xb9\xc5\xed\xbc?u\xa2\xfd\x11cY\xa3?\x801i2}\xdd\x9c?\xd1@\xbc\rxG\xd1?b\xd1\xaba\x89\xaf\xd4?\xe6\xb4U\r\xfa\x92\xc4?a\xfb\xab\xbb\xf1\xa5\xce?\x84\xa7\x0eW\xf8\xda\xc4?\xa0_\xb4\xa1\xa7R\xb2?\x00\x00\x00\x00\x00\x00\x00\x00\xfb\xb5\xf3\x1buF\xc0?\xe0\xa8\x05b\xac\x99\xaa?\xf2\x1e=w\xe9\xc6\xc7?m\x97\xd58\xee\xb4\xc9?\xc0\xa8\x0f\xcb\xcdU\xb7?\x16@C\xd8\xedM\xa8?\x00Z\x05\x1e\xfc\xd2\x7f?qFtqGba]qHh\nh\x0b}qI(h\rh\x0eh\x0fK\x00\x85qJh\x11\x87qKRqL(K\x01K\x05\x85qMh\x18\x89]qN(h\x1ch\x1dh\x1eh\x1fh etqObh"Nu\x86qPRqQa}qRX\x06\x00\x00\x000.14.1qS}qT(X\x04\x00\x00\x00axesqUh\tX\x06\x00\x00\x00blocksqV]qW}qX(X\x06\x00\x00\x00valuesqYh?X\x08\x00\x00\x00mgr_locsqZcbuiltins\nslice\nq[K\x00K\x05K\x01\x87q\\Rq]uaustq^bX\x04\x00\x00\x00_typq_X\t\x00\x00\x00dataframeq`X\t\x00\x00\x00_metadataqa]qbX\x05\x00\x00\x00attrsqc}qdubas.'

---

**R1236J** - 9


In [ ]:
# no relevant experiments for this subject

---

**R1391T** - 10


In [ ]:
byte_alpha = b'\x80\x03}q\x00X\x06\x00\x00\x00R1391Tq\x01]q\x02(cpandas.core.frame\nDataFrame\nq\x03)\x81q\x04}q\x05(X\x05\x00\x00\x00_dataq\x06cpandas.core.internals.managers\nBlockManager\nq\x07)\x81q\x08(]q\t(cpandas.core.indexes.base\n_new_Index\nq\ncpandas.core.indexes.base\nIndex\nq\x0b}q\x0c(X\x04\x00\x00\x00dataq\rcnumpy.core.multiarray\n_reconstruct\nq\x0ecnumpy\nndarray\nq\x0fK\x00\x85q\x10C\x01bq\x11\x87q\x12Rq\x13(K\x01K\x05\x85q\x14cnumpy\ndtype\nq\x15X\x02\x00\x00\x00O8q\x16K\x00K\x01\x87q\x17Rq\x18(K\x03X\x01\x00\x00\x00|q\x19NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?tq\x1ab\x89]q\x1b(X\x05\x00\x00\x00countq\x1cX\r\x00\x00\x00mean_recalledq\x1dX\x11\x00\x00\x00mean_not_recalledq\x1eX\x0c\x00\x00\x00std_recalledq\x1fX\x10\x00\x00\x00std_not_recalledq etq!bX\x04\x00\x00\x00nameq"Nu\x86q#Rq$h\nh\x0b}q%(h\rh\x0eh\x0fK\x00\x85q&h\x11\x87q\'Rq((K\x01K\x10\x85q)h\x18\x89]q*(X\x0f\x00\x00\x00parahippocampalq+G\x7f\xf8\x00\x00\x00\x00\x00\x00X\x10\x00\x00\x00inferiortemporalq,X\x06\x00\x00\x00insulaq-X\x0e\x00\x00\x00middletemporalq.X\n\x00\x00\x00entorhinalq/X\x10\x00\x00\x00superiortemporalq0X\x08\x00\x00\x00fusiformq1X\x13\x00\x00\x00medialorbitofrontalq2X\x14\x00\x00\x00lateralorbitofrontalq3X\r\x00\x00\x00parsorbitalisq4X\x10\x00\x00\x00parstriangularisq5X\x18\x00\x00\x00rostralanteriorcingulateq6X\x14\x00\x00\x00rostralmiddlefrontalq7X\x10\x00\x00\x00inferiorparietalq8X\x0c\x00\x00\x00temporalpoleq9etq:bh"X\x04\x00\x00\x00areaq;u\x86q<Rq=e]q>h\x0eh\x0fK\x00\x85q?h\x11\x87q@RqA(K\x01K\x05K\x10\x86qBh\x15X\x02\x00\x00\x00f8qCK\x00K\x01\x87qDRqE(K\x03X\x01\x00\x00\x00<qFNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tqGb\x89B\x80\x02\x00\x00\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00A@\x00\x00\x00\x00\x00\x00*@\x00\x00\x00\x00\x00\x00$@\x00\x00\x00\x00\x00\x00.@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00&@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00@\xf1s\xb8mR\x8f\t\xc0\xc9e\xa62\x96\xa3\t\xc0\x0fpN\xf2\x80\xc4\n\xc0Ih)\xd9\\\x8d\x07\xc0WO\xea\x1c\xe4P\n\xc0 \xae\xc0\xf5:%\t\xc0\xe9#\xe84\x17]\t\xc08\x1ct\x98\xfb\xdc\n\xc0\x05p\x9d\n`\xf0\n\xc0pT\x8a\xaa\xc6\xa9\x07\xc0,Hu^\xeb\x18\x06\xc0\x89\x89\x7f!\x92\xe0\x08\xc0\x14\x84K~\xbc>\x08\xc0<\xf2S\x0f\x18\xc1\x06\xc0\xbda<YZ\xac\t\xc0\xf2o\xf1\x8d\x88S\x07\xc0\xa6\x01T\xba\xbb.\t\xc0\x1a;\xad\x0e$\xc7\t\xc0\xc1)M\xb6\xc4\xbf\n\xc0mK\x95\x06s\x92\x07\xc0\xa6\xc1\xb7\xff\x12J\n\xc0\xd9\xa8\xe7\r0d\t\xc0e\x07\x8a\xd6Zv\t\xc0\xe1\xdd\xf9\x94\x84\x85\x0b\xc0\xb0R\xe6O\x8e\xbb\x0b\xc0\xed\xcb\x85\x1a\xd0\x17\x08\xc0\x10\xb2\\/\xfel\x06\xc0\xfd\xd7\xa5V\x10\x1e\t\xc0U\xd0\x8b#]\xad\x08\xc0\x16Ig\x92\xba\xcc\x06\xc0b=_\xe2\xbbe\t\xc0\x0fC\xfb\xa12\x8a\x07\xc0\xf2\xfa\xe9\xed\x93\x0b\xb8?\xa11\x1a\x19\x02\xb6\xcf?@\xdc,R\xec\xa7\xc6?\x08\x9e\x07\xaa\xaf\xaf\xcb?\x8d5\x06\x12\xe3\x16\xcd?(\xb9\xfe\xe4\xbd\xdc\xca?\xf8\'xA\xb62\xc0?\xd8h\x0b\x02J\xb9\xce?\xaf!\xdd\xb7\x83\xd3\xc5?\x7f-j\xb8\xe7\xcf\xc9?\x94\x03K\x94\xd9\x04\xc8?\x00\x00\x00\x00\x00\x00\x00\x00\x1e\xcfDh&i\xb8?\xc3\xd2\xe9\x9a\xeb\x1a\xce?\x00\x00\x00\x00\x00\x00\x00\x00\xa0\xd5\x07\xc3\xf0R\xc1?+\x8f\xa51`F\xc2?\x0200 \x10\xf2\xcd?\x87\xd4LI\x81\xe9\xc6?\xd7!\x8f\xbc\xca\x86\xcb?\tqx0O\xb7\xce?R\x82]\xf1T\xc0\xc8?D\xc8\x18"\xb2G\xc0?\xe0\x1a\xd6\xda,1\xc9?N\x1e{\x01s\x87\xc9?j\xa0\xdc\xfd\xa8\xe6\xc8?\x97}\x89\x9c\xd7X\xc7?\x00\x00\x00\x00\x00\x00\x00\x00a\xcf?r\x98\xcb\xb9?\x81cMK\xc9\xb8\xcf?\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfbs\tp8\xc3?qHtqIba]qJh\nh\x0b}qK(h\rh\x0eh\x0fK\x00\x85qLh\x11\x87qMRqN(K\x01K\x05\x85qOh\x18\x89]qP(h\x1ch\x1dh\x1eh\x1fh etqQbh"Nu\x86qRRqSa}qTX\x06\x00\x00\x000.14.1qU}qV(X\x04\x00\x00\x00axesqWh\tX\x06\x00\x00\x00blocksqX]qY}qZ(X\x06\x00\x00\x00valuesq[hAX\x08\x00\x00\x00mgr_locsq\\cbuiltins\nslice\nq]K\x00K\x05K\x01\x87q^Rq_uaustq`bX\x04\x00\x00\x00_typqaX\t\x00\x00\x00dataframeqbX\t\x00\x00\x00_metadataqc]qdX\x05\x00\x00\x00attrsqe}qfubh\x03)\x81qg}qh(h\x06h\x07)\x81qi(]qj(h\nh\x0b}qk(h\rh\x0eh\x0fK\x00\x85qlh\x11\x87qmRqn(K\x01K\x05\x85qoh\x18\x89]qp(h\x1cX\r\x00\x00\x00mean_recalledqqX\x11\x00\x00\x00mean_not_recalledqrX\x0c\x00\x00\x00std_recalledqsX\x10\x00\x00\x00std_not_recalledqtetqubh"Nu\x86qvRqwh\nh\x0b}qx(h\rh\x0eh\x0fK\x00\x85qyh\x11\x87qzRq{(K\x01K\x10\x85q|h\x18\x89]q}(h+G\x7f\xf8\x00\x00\x00\x00\x00\x00h,h-h.h/h0h1h2h3h4h5h6h7h8h9etq~bh"h;u\x86q\x7fRq\x80e]q\x81h\x0eh\x0fK\x00\x85q\x82h\x11\x87q\x83Rq\x84(K\x01K\x05K\x10\x86q\x85hE\x89B\x80\x02\x00\x00\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00A@\x00\x00\x00\x00\x00\x00*@\x00\x00\x00\x00\x00\x00$@\x00\x00\x00\x00\x00\x00.@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00&@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00@n \xb2\x9bYn\x07\xc0?\x16\xf2\xd2F\xd2\x06\xc0\x99\x16\x9a\r\xfc:\x08\xc0i~m\x9e\n\x1a\x06\xc0\xdeO\x074\x06\xa0\x07\xc0p+\xd5\xf6\xea\xc2\x07\xc0I\x8cY\xb9\xbb\xcb\x07\xc0\xfc\x92\xca{\xe3&\x07\xc0\xf3\x0f\x15\x81\x07\x84\x07\xc06\x93q}<I\x05\xc0\xd7\x04\x89xs\t\x02\xc0\xf9\xaa\x863\xbaQ\x05\xc0{u\x18\x9bNb\x06\xc0\x9a\x97\xb5`#~\x06\xc0\x97\xb2r\xcaT\xe3\x07\xc0m\x13:\x96\x16\xc5\x05\xc0\x98asXPw\x07\xc0}r\x14\xc9\xdcg\x07\xc0i<lC7\xea\x08\xc0\x0b^\xd0`\xb7:\x06\xc0\x8a\x0bC\xfa\xa6$\x08\xc0\x149Ok\x85\xed\x07\xc0\x08iK\xc4_c\x08\xc0\xd0\x82y\xac\xc9\x91\x07\xc0\xd0\xf7\x15\xeeI\xfb\x07\xc09\xe0\xf6vD\xbe\x05\xc0G\xd1\x80Y\xcay\x02\xc0\x07O\x93\x01fn\x05\xc0\xb1\xdf\xba\xae\xf1\xa4\x06\xc0\x03\xc1Va\x93\xe1\x06\xc0e\xd9\x13xex\x08\xc0\x94\'f\xac+9\x06\xc0\x02\xafMqL\xcc\xb8?\x99\x8d\xc7\x93\x89Z\xd2?|u\x11\xd8\t\xb1\xc9?D\x10\xc7\xf7\x05\xa5\xc7?\xc4[\xb9\x81.j\xd3?\xbd\xdf\xc9N\xcd\x0b\xd0?\x84\xe1\x00\xf1Q\xcc\xbc?p\x12\x00\xb0\xe5l\xcf?\xbb\xfd`\xaa\x7fA\xbe?Q\xb9+\x9aC6\xdb?\x0b\x08\x7f\x05\xda0\xc3?\x00\x00\x00\x00\x00\x00\x00\x00\xe5\xc0\xecO\x1b\x97\xb6?H\xb4b\xde\x85\x13\xc6?\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfcD\xd8F"\x9f?m^\x9967\x11\xb7?AK=\x8dR\x9d\xd2?I\x8c\x8e\xe7{\xd1\xc7?\x0f\xe7~\xc2\x9f)\xc9?3\xe4\x9f\xb3-\x92\xd2?\xfe\xc2\x8dYV\xe5\xca?\x8c[\x0b\x00"\xcc\xbc?\xb0\xa61\x00\x7f*\xcd?\x9bY^t4\xbb\xc2?\xf1\xf7\x91\xe3\xfaG\xda?\x7f\xb8\xb9\xfd@\x9d\xc2?\x00\x00\x00\x00\x00\x00\x00\x00\xb6"\x00!nd\xb8?)\x86\xa16\x154\xc3?\x00\x00\x00\x00\x00\x00\x00\x00\xa0\xf3\xef0\x16\xfc\xa7?q\x86tq\x87ba]q\x88h\nh\x0b}q\x89(h\rh\x0eh\x0fK\x00\x85q\x8ah\x11\x87q\x8bRq\x8c(K\x01K\x05\x85q\x8dh\x18\x89]q\x8e(h\x1chqhrhshtetq\x8fbh"Nu\x86q\x90Rq\x91a}q\x92hU}q\x93(hWhjhX]q\x94}q\x95(h[h\x84h\\h]K\x00K\x05K\x01\x87q\x96Rq\x97uaustq\x98bhahbhchdhe}q\x99ubh\x03)\x81q\x9a}q\x9b(h\x06h\x07)\x81q\x9c(]q\x9d(h\nh\x0b}q\x9e(h\rh\x0eh\x0fK\x00\x85q\x9fh\x11\x87q\xa0Rq\xa1(K\x01K\x05\x85q\xa2h\x18\x89]q\xa3(h\x1cX\r\x00\x00\x00mean_recalledq\xa4X\x11\x00\x00\x00mean_not_recalledq\xa5X\x0c\x00\x00\x00std_recalledq\xa6X\x10\x00\x00\x00std_not_recalledq\xa7etq\xa8bh"Nu\x86q\xa9Rq\xaah\nh\x0b}q\xab(h\rh\x0eh\x0fK\x00\x85q\xach\x11\x87q\xadRq\xae(K\x01K\x10\x85q\xafh\x18\x89]q\xb0(h+G\x7f\xf8\x00\x00\x00\x00\x00\x00h,h-h.h/h0h1h2h3h4h5h6h7h8h9etq\xb1bh"h;u\x86q\xb2Rq\xb3e]q\xb4h\x0eh\x0fK\x00\x85q\xb5h\x11\x87q\xb6Rq\xb7(K\x01K\x05K\x10\x86q\xb8hE\x89B\x80\x02\x00\x00\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00A@\x00\x00\x00\x00\x00\x00*@\x00\x00\x00\x00\x00\x00$@\x00\x00\x00\x00\x00\x00.@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00&@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00@,\x9e\x17\xbb \xb9\x07\xc0\xbc\xce\x10f\xedn\x07\xc07\xdb3\xc4+\xc1\x08\xc0\x06|1~I\x85\x06\xc0\xf2!\x82W\xab!\x08\xc0\x99h\x86U\xed_\x08\xc0\x9b\xa1\x0e\x08Oa\x08\xc0\xfa\xbaI\xd6K\x97\x07\xc0\x95U\x80/\x13\xbe\x07\xc0\x0f\xa8\xac\xf1\xe2\xb1\x04\xc0\xa4\x0fE\xb3}H\x02\xc0@e\xb7\xb7\xd6U\x05\xc0\x1f\xeaP2\x98\xc8\x06\xc0\xa7\'\ni\xda\x16\x05\xc0\xd6\x11\xbd!wr\x08\xc0\xe5J\xdf\x08\xc4z\x06\xc0o\x16\x9de\xb0\xdb\x07\xc0\xed\xdeh\xf4\x8c\x9c\x07\xc0\xea\x19\xad\xfe\xc8\x19\t\xc0/\x98\x16\xa1\xdfv\x06\xc0\xc3\nQ\x16b:\x08\xc0\x80\xa9\x950\xe6\x1d\x08\xc0\xcc\xe57\x8a8e\x08\xc0\x0b\xc5V9\x1e\x96\x07\xc0\xe4\x10\x88\xb2\xff\x08\x08\xc0\xa4\x8f\xd4\x03:\xf2\x04\xc0\xc4\xe9\xe1\xfc\x91\x94\x02\xc0\xe3\xf0\xa7\x92g\xc5\x05\xc0Wf;*\xd1\xae\x06\xc05\xcb\x00\xbesL\x05\xc0%.\xd1\xbeo\x98\x08\xc0\xd2{<\x85\x1d}\x06\xc0m|\xf2\t\xfe0\xb8?\x12\xe6v\x7f*m\xd2?/\xa4\x04\xab\xb0\x8c\xc9?\xad\x95obT\x8b\xc8?\x9b\x8e\x83\xb2\x95"\xd3?\x0f\xd0\xb8\xe8i\xf4\xca?\xe0\xcc\xe2\xbdo\x9b\xbe?8F\xf5N\xf6\x99\xcc?C\xc0\x87\t\xb5\x03\xc0?\xd2\x81}\xc0\x1c\x9b\xd1?\xcd\xb1h\x02\xb7\x10\xc5?\x00\x00\x00\x00\x00\x00\x00\x00\xbc\xfe<T#\x06\xb7?(\x15Ua\x19r\xc1?\x00\x00\x00\x00\x00\x00\x00\x00@\xe5W\x1a\xb4\x1c\xa1?\xfb\xcbA\xae^\x0f\xba?\xf1v\xebW\x14 \xd2?p\xec\xf2\xc8\x85\x8f\xc8?\xc4\x1a\x1cB\xed\x19\xc8?\x8b\xa8\xabD.\xb2\xd1?\xc46\xe4\x8c\xd1\xd4\xcb?\xb5#>\x91\xba\xef\xbe?\xc0*\x84V\x859\xc1?\xb8\x94*\xd2[a\xc2?p\x0e\x00\xa3\xac1\xd0?\xa6y\xd19\x02\x16\xc3?\x00\x00\x00\x00\x00\x00\x00\x00\xa5K\x1e\xa9\x13\xe1\xb9?\x13\xcfW\x93t\xcc\xc1?\x00\x00\x00\x00\x00\x00\x00\x00`5q\x8c\x1e\xa7\xa1?q\xb9tq\xbaba]q\xbbh\nh\x0b}q\xbc(h\rh\x0eh\x0fK\x00\x85q\xbdh\x11\x87q\xbeRq\xbf(K\x01K\x05\x85q\xc0h\x18\x89]q\xc1(h\x1ch\xa4h\xa5h\xa6h\xa7etq\xc2bh"Nu\x86q\xc3Rq\xc4a}q\xc5hU}q\xc6(hWh\x9dhX]q\xc7}q\xc8(h[h\xb7h\\h]K\x00K\x05K\x01\x87q\xc9Rq\xcauaustq\xcbbhahbhchdhe}q\xccubes.'

---

**R1401J** - 11 


In [ ]:
byte_alpha = b'\x80\x03}q\x00X\x06\x00\x00\x00R1401Jq\x01]q\x02cpandas.core.frame\nDataFrame\nq\x03)\x81q\x04}q\x05(X\x05\x00\x00\x00_dataq\x06cpandas.core.internals.managers\nBlockManager\nq\x07)\x81q\x08(]q\t(cpandas.core.indexes.base\n_new_Index\nq\ncpandas.core.indexes.base\nIndex\nq\x0b}q\x0c(X\x04\x00\x00\x00dataq\rcnumpy.core.multiarray\n_reconstruct\nq\x0ecnumpy\nndarray\nq\x0fK\x00\x85q\x10C\x01bq\x11\x87q\x12Rq\x13(K\x01K\x05\x85q\x14cnumpy\ndtype\nq\x15X\x02\x00\x00\x00O8q\x16K\x00K\x01\x87q\x17Rq\x18(K\x03X\x01\x00\x00\x00|q\x19NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?tq\x1ab\x89]q\x1b(X\x05\x00\x00\x00countq\x1cX\r\x00\x00\x00mean_recalledq\x1dX\x11\x00\x00\x00mean_not_recalledq\x1eX\x0c\x00\x00\x00std_recalledq\x1fX\x10\x00\x00\x00std_not_recalledq etq!bX\x04\x00\x00\x00nameq"Nu\x86q#Rq$h\nh\x0b}q%(h\rh\x0eh\x0fK\x00\x85q&h\x11\x87q\'Rq((K\x01K\x0f\x85q)h\x18\x89]q*(X\x14\x00\x00\x00lateralorbitofrontalq+X\x14\x00\x00\x00rostralmiddlefrontalq,G\x7f\xf8\x00\x00\x00\x00\x00\x00X\x10\x00\x00\x00parstriangularisq-X\x0f\x00\x00\x00superiorfrontalq.X\x06\x00\x00\x00insulaq/X\x0f\x00\x00\x00parsopercularisq0X\x13\x00\x00\x00caudalmiddlefrontalq1X\n\x00\x00\x00precentralq2X\x0e\x00\x00\x00middletemporalq3X\x10\x00\x00\x00inferiortemporalq4X\x10\x00\x00\x00superiortemporalq5X\x0b\x00\x00\x00postcentralq6X\x12\x00\x00\x00posteriorcingulateq7X\x0b\x00\x00\x00paracentralq8etq9bh"X\x04\x00\x00\x00areaq:u\x86q;Rq<e]q=h\x0eh\x0fK\x00\x85q>h\x11\x87q?Rq@(K\x01K\x05K\x0f\x86qAh\x15X\x02\x00\x00\x00f8qBK\x00K\x01\x87qCRqD(K\x03X\x01\x00\x00\x00<qENNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tqFb\x89BX\x02\x00\x00\x00\x00\x00\x00\x00\x000@\x00\x00\x00\x00\x00\x00,@\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00*@\x00\x00\x00\x00\x00\x00*@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00(@\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf0?\xc2\xd0\xf2\x9a\x01\xa1\x06\xc0\xad\x1fa(~\x9a\x06\xc06\x87\x80\xda\xccT\x08\xc0\x87?d< F\x05\xc0BM\xc4G\xdd\xee\x06\xc0\x89\r}\x1c\x81\xd3\x08\xc0\x9f\x9a\xfe\xcbl5\t\xc0a\xa0M8\xc6\xe6\x04\xc0\xfb\xfa\xb5\x05\x8bq\x07\xc0\xa9\xb6\xa1K\xf5\x15\n\xc0@HJ\xa1n\x1e\x0c\xc0\x8b\x8f\xe2Y4\xa7\t\xc0\xef^\xfc\x7f\x10\x1f\t\xc0\xd4Rc\xc7\xa7\xfa\n\xc0#\x1f\x07%.\xe3\x06\xc0\xc7J\xc6\xffm\xb4\x06\xc0\x18,\xa8\xbb$\xcc\x06\xc0\xc1\x89\xdf\x1b\xad\xa7\x08\xc0\x02\xaf\xd7\x81\xc2y\x05\xc0\xf2?\xa4\xa4\xdb/\x07\xc0\x199b\xf9,\xb6\x08\xc0r\xcf\x8c6\xc89\t\xc0\xdfY\\\x1f\x0f\x1a\x05\xc0p\xb5Oox_\x07\xc0z \xbdR\xfe\x1d\n\xc0W\xcbGv\xc1\x0b\x0c\xc0\x99\xba#\xa8\x1d"\n\xc0\xfd\x95\x96\xf1\xbct\t\xc0*P}\x16\xbc\r\x0b\xc0\x80\x05t\xa6\xd9"\x07\xc03-3\xae;M\xd9?\r\xdf\x87\xcb\xbak\xc8?\xf9\x84\xdd`\\\xb8\xce?\x00\x00\x00\x00\x00\x00\x00\x002\xf8\xdb\xeaM.\xd1?"a\x8b\x06\x0f4\xd0?\x00\x00\x00\x00\x00\x00\x00\x00\x8c\xdd\xe2]\x149\xd1?\x05\xa1`\xf5\x9b\xe4\xd8?\x97E\x02\xb6y"\xbe?\x00\x00\x00\x00\x00\x00\x00\x00\xb9\x9c\xfcr\x92M\xb3?O\x0b\xc1\x02pG\xc7?\x00\xdc\xabu\x9f\xe5R?\x00\x00\x00\x00\x00\x00\x00\x00\x1e\xd2\'E\xb5\xee\xd8?p\xbc\xa7\xe5\x8f\x8b\xc8?,CZ\\\xdf\x03\xcf?\x00\x00\x00\x00\x00\x00\x00\x00\xd0\x0c\xa6Q\xd9\xdf\xcf?+\x1b\xa4>\xbf\xc2\xc8?\x00\x00\x00\x00\x00\x00\x00\x00\xfd(H\x14 \x91\xd0?\x93\xde<Hw\x8e\xd5?\r\x19\x07\xd4\xe9C\xbd?\x00\x00\x00\x00\x00\x00\x00\x00\xa4n\xa94\x1d\xaa\xb8?\x06nt\xce\xfb\x1a\xc6?\xe0^;\xf6\xf8\xee\xa1?\x00\x00\x00\x00\x00\x00\x00\x00qGtqHba]qIh\nh\x0b}qJ(h\rh\x0eh\x0fK\x00\x85qKh\x11\x87qLRqM(K\x01K\x05\x85qNh\x18\x89]qO(h\x1ch\x1dh\x1eh\x1fh etqPbh"Nu\x86qQRqRa}qSX\x06\x00\x00\x000.14.1qT}qU(X\x04\x00\x00\x00axesqVh\tX\x06\x00\x00\x00blocksqW]qX}qY(X\x06\x00\x00\x00valuesqZh@X\x08\x00\x00\x00mgr_locsq[cbuiltins\nslice\nq\\K\x00K\x05K\x01\x87q]Rq^uaustq_bX\x04\x00\x00\x00_typq`X\t\x00\x00\x00dataframeqaX\t\x00\x00\x00_metadataqb]qcX\x05\x00\x00\x00attrsqd}qeubas.'

---

**R1361C** - 12


In [ ]:
byte_alpha = b'\x80\x03}q\x00X\x06\x00\x00\x00R1361Cq\x01]q\x02(cpandas.core.frame\nDataFrame\nq\x03)\x81q\x04}q\x05(X\x05\x00\x00\x00_dataq\x06cpandas.core.internals.managers\nBlockManager\nq\x07)\x81q\x08(]q\t(cpandas.core.indexes.base\n_new_Index\nq\ncpandas.core.indexes.base\nIndex\nq\x0b}q\x0c(X\x04\x00\x00\x00dataq\rcnumpy.core.multiarray\n_reconstruct\nq\x0ecnumpy\nndarray\nq\x0fK\x00\x85q\x10C\x01bq\x11\x87q\x12Rq\x13(K\x01K\x05\x85q\x14cnumpy\ndtype\nq\x15X\x02\x00\x00\x00O8q\x16K\x00K\x01\x87q\x17Rq\x18(K\x03X\x01\x00\x00\x00|q\x19NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?tq\x1ab\x89]q\x1b(X\x05\x00\x00\x00countq\x1cX\r\x00\x00\x00mean_recalledq\x1dX\x11\x00\x00\x00mean_not_recalledq\x1eX\x0c\x00\x00\x00std_recalledq\x1fX\x10\x00\x00\x00std_not_recalledq etq!bX\x04\x00\x00\x00nameq"Nu\x86q#Rq$h\nh\x0b}q%(h\rh\x0eh\x0fK\x00\x85q&h\x11\x87q\'Rq((K\x01K\x0e\x85q)h\x18\x89]q*(X\x06\x00\x00\x00insulaq+X\x10\x00\x00\x00superiortemporalq,X\x0e\x00\x00\x00middletemporalq-G\x7f\xf8\x00\x00\x00\x00\x00\x00X\n\x00\x00\x00entorhinalq.X\x08\x00\x00\x00fusiformq/X\x0f\x00\x00\x00parahippocampalq0X\x10\x00\x00\x00inferiortemporalq1X\x12\x00\x00\x00transversetemporalq2X\x17\x00\x00\x00caudalanteriorcingulateq3X\x0f\x00\x00\x00superiorfrontalq4X\x14\x00\x00\x00lateralorbitofrontalq5X\x13\x00\x00\x00caudalmiddlefrontalq6X\x12\x00\x00\x00posteriorcingulateq7etq8bh"X\x04\x00\x00\x00areaq9u\x86q:Rq;e]q<h\x0eh\x0fK\x00\x85q=h\x11\x87q>Rq?(K\x01K\x05K\x0e\x86q@h\x15X\x02\x00\x00\x00f8qAK\x00K\x01\x87qBRqC(K\x03X\x01\x00\x00\x00<qDNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tqEb\x89B0\x02\x00\x00\x00\x00\x00\x00\x00\x00B@\x00\x00\x00\x00\x00\x00$@\x00\x00\x00\x00\x00\x001@\x00\x00\x00\x00\x00\x00(@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xf0?\x14\x18\x86\xed\xe5\x01\x08\xc0\xde\x82\xc7@\xb3\xeb\x06\xc0\x02D\x1a\x02\xdf\xce\t\xc0\xcdO\x03\xc5\xa1\x1a\t\xc0C6\x10T\xd9;\t\xc0\x80v\x17\xfcJ\xab\t\xc0\x1cZQI\x8d\xe0\x06\xc0w\xb4I\xf7#\x12\x08\xc0\x01\xe4z\xd4\xef\x88\x06\xc0\x19j=\xcf\xc8\xde\x06\xc0\xa3n\xd5\xf18\xff\x07\xc0\xe8m2B\x99\xf7\x06\xc0S\x14q>@\x88\x08\xc06z\x96\xe1\x80q\x07\xc0\xdd\xe3jN\xf4\xeb\x07\xc03G\xe9"x\n\x07\xc0?F0\x15\xac\xd4\t\xc0\x11\x1e\xce\xb77\x04\t\xc0{\xb6\xcf\x8c\xd2*\t\xc0D\x1c\x00H/\xb1\t\xc0]\xbf\xe1\x02\xb38\x06\xc0\xa7\xae1\xa3\xf1\xff\x07\xc0+:\x91\x92Qr\x06\xc0\rR\xf5"\xd0\xc5\x06\xc0\n\xb8\xa8\xc0\xb6\x95\x07\xc0\x83\xc8I\x97\x03\xba\x06\xc0+\xb3\'hJW\x08\xc0V\xa8\xf2\xfa\x9c\x7f\x07\xc0J\xe1\x11*\x1c\xf0\xc6?\xb6&\x87\xb9\xbc\xa1\xde?\x8f\r\x8cF\r"\xd1?\xd6\xadZjDC\xd2?}\x81\xa1\x01F\xb6\xc7?\x91\x01}5\xe02\xcd?\x8co\xd5\xe6\xaf\x86\xc9?\xc4\xcfU\x8cY\xe6\xc7?\x8a!\xdd\xeb\xc7\xb0\xd5?\xc6\xff"}d\x97\xb0?T\n\xa2\x83/S\xb4?7\x8c#\x1f+\x84\xc1?\xdb\xdb \x88\xedP\xb8?\x00\x00\x00\x00\x00\x00\x00\x00},F\x96\xd9\x18\xc8?\xc5g\xda\x05\xcc\xf3\xdd?\x0b\xc4C\xfd\xf0\\\xcf?X\x9e\x8b`/\xb7\xd0?\x04\x05\xe80\xb3\xf4\xca?\xb0\xe7I\xcf\x0b\x90\xca?\x05\xd3\x1b\x95\xf6y\xd0?\xe6\x9d\rpR\xaa\xc7?J\xb7\x9d=\xfd\x01\xd7?b\xba\x8e\xc5z\xdb\xb2?\x1a71K\x16\xb3\xb5?\xbe:\x00v\x91\x17\xc1?\xa4\x87\x8a\xde\x1e\xa2\xb9?\x00\x00\x00\x00\x00\x00\x00\x00qFtqGba]qHh\nh\x0b}qI(h\rh\x0eh\x0fK\x00\x85qJh\x11\x87qKRqL(K\x01K\x05\x85qMh\x18\x89]qN(h\x1ch\x1dh\x1eh\x1fh etqObh"Nu\x86qPRqQa}qRX\x06\x00\x00\x000.14.1qS}qT(X\x04\x00\x00\x00axesqUh\tX\x06\x00\x00\x00blocksqV]qW}qX(X\x06\x00\x00\x00valuesqYh?X\x08\x00\x00\x00mgr_locsqZcbuiltins\nslice\nq[K\x00K\x05K\x01\x87q\\Rq]uaustq^bX\x04\x00\x00\x00_typq_X\t\x00\x00\x00dataframeq`X\t\x00\x00\x00_metadataqa]qbX\x05\x00\x00\x00attrsqc}qdubh\x03)\x81qe}qf(h\x06h\x07)\x81qg(]qh(h\nh\x0b}qi(h\rh\x0eh\x0fK\x00\x85qjh\x11\x87qkRql(K\x01K\x05\x85qmh\x18\x89]qn(h\x1cX\r\x00\x00\x00mean_recalledqoX\x11\x00\x00\x00mean_not_recalledqpX\x0c\x00\x00\x00std_recalledqqX\x10\x00\x00\x00std_not_recalledqretqsbh"Nu\x86qtRquh\nh\x0b}qv(h\rh\x0eh\x0fK\x00\x85qwh\x11\x87qxRqy(K\x01K\x0e\x85qzh\x18\x89]q{(h+h,h-G\x7f\xf8\x00\x00\x00\x00\x00\x00h.h/h0h1h2h3h4h5h6h7etq|bh"h9u\x86q}Rq~e]q\x7fh\x0eh\x0fK\x00\x85q\x80h\x11\x87q\x81Rq\x82(K\x01K\x05K\x0e\x86q\x83hC\x89B0\x02\x00\x00\x00\x00\x00\x00\x00\x00B@\x00\x00\x00\x00\x00\x00$@\x00\x00\x00\x00\x00\x001@\x00\x00\x00\x00\x00\x00(@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xf0?\xec\xb2\xa4-,\\\x08\xc0z\xa0;BqC\x07\xc0\x83\x98\x1f\xb4\x12\xe5\t\xc0\xb3\xfc\x13&\x17\x8f\t\xc0\\\x04,\x12\xbf\x89\x08\xc0l\x9b\xed\x95=E\n\xc08%Nq\xbd\xc2\x06\xc0\xf2\xef{\x17B\xb8\x08\xc0\xa7\xcf\xab\x1eL\xa5\x06\xc0\x7fR\xa2\x03\x94b\x07\xc0\xe7+\xe63\xc8\t\x08\xc0OUm\xc7C7\x07\xc0\x07r\x85\xc9?\xfa\x08\xc0"\xe0\xfa\x98\xff\x8b\x07\xc0\x99i\xbdjF\xd2\x08\xc0\xe6y@\x9e\xf1\x95\x08\xc0Cc\xe8\xad\x08\xf7\t\xc0\xbb\xfe\xbe\xfbU\x17\n\xc0H\xf1:F\xc9j\x08\xc0\xc7/\xb8\xc8\xadu\n\xc0\xb3\xfcR\xf7\xdf\xdc\x06\xc0Z\xbd\x9c\xe8\x87~\x08\xc0\xe8\x17F\x86/\xc5\x07\xc0q\xeb\x11]\r\x92\x07\xc0O\x03,d\xa4\x07\x08\xc0\xfc\r\xf9\xad3\x19\x07\xc0B\x10\xa0\x91\xad\xd8\x08\xc0\xfe\xa7\xf49$\xa2\x07\xc0E\xf4\xd3\xf5n\x02\xc4?\x80\xcbR\x92\x11\xbb\xde?g\xaap\x83\xa3#\xd0?\xcb\r\x9d\x9b\xbc\xb2\xce?gQc\x89e\x99\xc1?ba\xa8\x17\x93\xda\xcb?\x9e\x80\xcf\x7f|\xd2\xd3?\xe1\xa5\x9b\xd6\xbd\xd0\xc1?c\xf3D<\xf5\x12\xd7?\nZ\xe3\xa8rT\xb5?e\xd8\xf6\xdf\xd4h\xb1?t\xedv\xff\xd3\x86\xc2?\xd7\xf7,\xa04\xbc\xbc?\x00\x00\x00\x00\x00\x00\x00\x00\xf34\xec\xbf\xd3\x01\xcf?#\x19\xb7&\x86T\xdc?\x9c\x0f\xd2Fn\xc4\xce?\xa8\\FF\xf0y\xce?\xfc\x85\x02\xd8{)\xc2?\x81\xbe\x96\x04\xab&\xd0?\xdf\xd7\x12\xde\xf0N\xd3?\xe8\xa7N9\x07n\xc3?\xf8\xd6\x98\xc5>\x0e\xe1?\xdeS\xe3=I\x1a\xb1?\x16\xc6\xbd\x8a\xc9\xf6\xb1?j\x94b\xb4p\xf8\xc2?[\x84\x1d\x1ea\xf1\xbb?\x00\x00\x00\x00\x00\x00\x00\x00q\x84tq\x85ba]q\x86h\nh\x0b}q\x87(h\rh\x0eh\x0fK\x00\x85q\x88h\x11\x87q\x89Rq\x8a(K\x01K\x05\x85q\x8bh\x18\x89]q\x8c(h\x1chohphqhretq\x8dbh"Nu\x86q\x8eRq\x8fa}q\x90hS}q\x91(hUhhhV]q\x92}q\x93(hYh\x82hZh[K\x00K\x05K\x01\x87q\x94Rq\x95uaustq\x96bh_h`hahbhc}q\x97ubes.'

---

**R1060M** - 13


In [ ]:
byte_alpha =  

# {D} running on one subject - divided to recalled/not

In [ ]:
index_df = load.Index()
subjects = index_df.subject.unique()
exp = 'FR1'
subject_number = 3

# will eventually be - for sub in subjects:
sub = subjects[subject_number]
df_select = index_df[(index_df['subject']==sub) & (index_df['experiment']==exp)]
channels_df = load.Load(df_select.iloc[0], 'channels')
eeg_gen = load_eeg(index_df, sub, exp)

# will eventually be - for epochs, events in eeg_gen:
epochs, events = next(eeg_gen)

sr = epochs.info['sfreq']

word_events = events[events['type']=='WORD']
word_idx = word_events.index
recalled_idx = word_events[word_events['recalled']==1].index
not_recalled_idx = word_events[word_events['recalled']==0].index

recalled_data = np.nan_to_num(epochs.get_data())[recalled_idx]
not_recalled_data = np.nan_to_num(epochs.get_data())[not_recalled_idx]
# print(recalled_data.shape)
# print(not_recalled_data.shape)

################################################################################

# We'd probably want to notch [60,120] at least
filtering_freqs = [60]

rec_filtered_eeg = apply_notch_filters(recalled_data, filtering_freqs, sr)
n_rec_filtered_eeg = apply_notch_filters(not_recalled_data, filtering_freqs, sr)
# print(rec_filtered_eeg.shape)
# print(n_rec_filtered_eeg.shape)

################################################################################

# electrodes should be a specific list per patient (i suggest dict((sub, region_names)))
# names = elec_name_per_subj_dict[sub] # a list of area names
electrodes = get_channel_indices(channels_df['ind.region'])

chunk_size = 10
# this condition prevents an addition of a null range
if ~len(electrodes) % chunk_size:
    chunks = [electrodes[chunk_size*i:chunk_size*(i+1)] for i in range(int(len(electrodes)/chunk_size))]
else:
    chunks = [electrodes[chunk_size*i:chunk_size*(i+1)] for i in range(int(len(electrodes)/chunk_size)+1)]
# chunks.append(electrodes[chunk_size*i:-1])

rec_log_morlet_eeg = get_log_morlet_generator(rec_filtered_eeg, sr, chunks)
n_rec_log_morlet_eeg = get_log_morlet_generator(n_rec_filtered_eeg, sr, chunks)

# iterating over data to get recalled and not recalled alpha values
rec_alphas_dict = {}
for elecs, rec_lmeeg in zip(chunks, rec_log_morlet_eeg):
    rec_alphas = [i[0] for i in get_alphas(rec_lmeeg).tolist()]
    # print(*zip(channels_df['ind.region'].iloc[elecs], rec_alphas))
    for ch_name, alpha in zip(channels_df['ind.region'].iloc[elecs], rec_alphas):
        if not ch_name in rec_alphas_dict.keys(): rec_alphas_dict[ch_name] = [alpha]
        else: rec_alphas_dict[ch_name].append(alpha)

for k,v in rec_alphas_dict.items(): rec_alphas_dict[k] = np.array(v)
# print(rec_alphas_dict)

n_rec_alphas_dict = {}
for elecs, n_rec_lmeeg in zip(chunks, n_rec_log_morlet_eeg):
    n_rec_alphas =  [i[0] for i in get_alphas(n_rec_lmeeg).tolist()]
    # print(*zip(channels_df['ind.region'].iloc[elecs], n_rec_alphas))
    for ch_name, alpha in zip(channels_df['ind.region'].iloc[elecs], n_rec_alphas):
        if not ch_name in n_rec_alphas_dict.keys(): n_rec_alphas_dict[ch_name] = [alpha]
        else: n_rec_alphas_dict[ch_name].append(alpha)

for k,v in n_rec_alphas_dict.items(): n_rec_alphas_dict[k] = np.array(v)        
# print(n_rec_alphas_dict)

################################################################################

rec_alphas_df = pd.DataFrame()
for k,v in rec_alphas_dict.items():
    rec_alphas_df = rec_alphas_df.append({'area':k, 'count':len(v), 
                                          'mean':v.mean(), 'std':v.std()}, 
                                         ignore_index=True)

rec_alphas_df = rec_alphas_df.set_index('area')
# rec_alphas_df

n_rec_alphas_df = pd.DataFrame()
for k,v in n_rec_alphas_dict.items():
    n_rec_alphas_df = n_rec_alphas_df.append({'area':k, 'count':len(v), 
                                              'mean':v.mean(), 'std':v.std()}, 
                                             ignore_index=True)

n_rec_alphas_df = n_rec_alphas_df.set_index('area')
# n_rec_alphas_df

################################################################################

# combine results for recalled and not recalled 
alpha_df = rec_alphas_df.join(n_rec_alphas_df, lsuffix='_recalled', rsuffix='_not_recalled')
alpha_df = alpha_df.rename(columns={'count_recalled':'count'}, inplace=False)
alpha_df = alpha_df.drop('count_not_recalled', axis=1)
ks = alpha_df.keys()
alpha_df = alpha_df[[ks[0], ks[1], ks[3], ks[2], ks[4]]]
alpha_df

found 4 matching FR1 experiments for subject R1377M
Not setting metadata
Not setting metadata
589 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
[Start filtering - 60Hz]
Setting up band-stop filter from 58 - 62 Hz

IIR filter parameters
---------------------
Butterworth bandstop zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 58.50, 61.50 Hz: -6.02, -6.02 dB

[Done filtering - 60Hz]
[Start filtering - 60Hz]
Setting up band-stop filter from 58 - 62 Hz

IIR filter parameters
---------------------
Butterworth bandstop zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 58.50, 61.50 Hz: -6.02, -6.02 dB

[Done filtering - 60Hz]
[Start WT, chunk - range(0, 10)]
[Start WT, chunk - range(10, 20)]
[Start WT, chunk - range(20, 30)]
[Start WT, chunk - range(30, 40)]
[Start WT, chunk - ran

,count,mean_recalled,mean_not_recalled,std_recalled,std_not_recalled
area,,,,,
insula,28.0,-2.853538,-2.852273,0.241304,0.255835
fusiform,3.0,-2.993009,-2.985582,0.195581,0.213358
inferiortemporal,3.0,-2.948659,-2.949247,0.037477,0.038666
middletemporal,17.0,-3.006214,-3.010030,0.215500,0.217513
NaN,10.0,-2.934228,-2.931943,0.182420,0.182889
unknown,1.0,-2.903693,-2.902142,0.000000,0.000000
caudalanteriorcingulate,4.0,-2.921076,-2.914771,0.236083,0.248498
superiorfrontal,11.0,-2.783920,-2.775061,0.203285,0.194619
caudalmiddlefrontal,9.0,-2.781431,-2.782305,0.226845,0.227328


# {D} running on one subject 

In [ ]:
index_df = load.Index()
subjects = index_df.subject.unique()
exp = 'FR1'

# will eventually be - for sub in subjects:
sub = subjects[1]
df_select = index_df[(index_df['subject']==sub) & (index_df['experiment']==exp)]
channels_df = load.Load(df_select.iloc[0], 'channels')
eeg_gen = load_eeg(index_df, sub, exp)

# will eventually be - for epochs, events in eeg_gen:
epochs, events = next(eeg_gen)
sr = epochs.info['sfreq']
word_events = events[events['type']=='WORD']
data = np.nan_to_num(epochs.get_data())[word_events.index]

In [ ]:
# We'd probably want to notch [60,120] at least
filtering_freqs = [60]
filtered_eeg = apply_notch_filters(data, filtering_freqs, sr)

In [ ]:
# electrodes should be a specific list per patient (i suggest dict((sub, electrodes)))
electrodes = range(55)

chunk_size = 10
chunks = [electrodes[chunk_size*i:chunk_size*(i+1)] for i in range(int(len(electrodes)/chunk_size) + 1)]
log_morlet_eeg = get_log_morlet_generator(filtered_eeg, sr, chunks)

# iterating over data to get recalled and not recalled alpha values

for elecs, lmeeg in zip(chunks, log_morlet_eeg):
    alphas = get_alphas(lmeeg).tolist()
    print(*zip(channels_df['ind.region'].iloc[elecs], alphas))